# FLIMLib Ops

## Dependencies

The FLIMJ ops live in `flimlib:flimj-ops`. This dependency has to be present in order to use the ops. You can either import the package from your maven local or the ImageJ central repository.

In [1]:
// uncomment to import from local repo
%classpath config resolver mvnLocal
// import from ImageJ central repo
%classpath config resolver scijava.public https://maven.scijava.org/content/groups/public
// uncomment to import from ImageJ central repo
%classpath add mvn flimlib flimj-ops 2.0.2
%classpath add mvn net.imagej imagej 2.0.0-rc-72

import net.imagej.ImageJ

ij = new ImageJ()
op = ij.op()
nb = ij.notebook()

Added new repo: mvnLocal
Added new repo: scijava.public


net.imagej.notebook.DefaultNotebookService [priority = 0.0]

In [2]:
// run this if dependency messed up
// %classpath reset

null

## Utility Code

Here is some utility code that helps display the multi-layer fitted images, no attention needed.

In [3]:
import net.imglib2.type.numeric.ARGBType
import net.imglib2.type.numeric.real.FloatType
import net.imagej.display.ColorTables
import net.imglib2.converter.Converters
import net.imglib2.converter.RealLUTConverter

class FancyDisplay {
    
    public channelAxis, op, nb
    
    public FancyDisplay(ij, channelAxis=3) {
        this.channelAxis = channelAxis
        this.op = ij.op()
        this.nb = ij.notebook()
    }
    
    public tableDisp(fitted, tMin=null, tMax=null, aMax=null, zMax=null) {
        def lifetimeAxis = fitted.ltAxis
        def fittedImg = fitted.paramMap
        def sampleZ = op.transform().hyperSliceView(fittedImg, lifetimeAxis, 0)
        def sampleA = []
        def sampleT = []
        for (int comp in 0..((fittedImg.dimension(lifetimeAxis) - 1) / 2 - 1)) {
            sampleA.push(op.transform().hyperSliceView(fittedImg, lifetimeAxis, comp * 2 + 1))
            sampleT.push(op.transform().hyperSliceView(fittedImg, lifetimeAxis, comp * 2 + 2))
        }

        println("Z min = " + op.stats().min(sampleZ))
        println("Z max = " + op.stats().max(sampleZ))
        for (int i in 0..sampleA.size() - 1) {
            println("A" + (i + 1) + " min = " + op.stats().min(sampleA[i]))
            println("A" + (i + 1) + " max = " + op.stats().max(sampleA[i]))
            println("Tau" + (i + 1) + " min = " + op.stats().min(sampleT[i]))
            println("Tau" + (i + 1) + " max = " + op.stats().max(sampleT[i]))
        }
        
        def pseudocolor = op.run("flim.showPseudocolor", fitted, tMin, tMax, 0, aMax);
        
        // default values from img
        zMax = zMax == null ? op.stats().max(sampleZ) : new FloatType(zMax)
        aMax = aMax == null ? op.stats().max(sampleA[0]) : new FloatType(aMax)
        tMin = tMin == null ? op.stats().min(sampleT[0]) : new FloatType(tMin)
        tMax = tMax == null ? op.stats().max(sampleT[0]) : new FloatType(tMax)
        
        def labeled = [:]
        labeled["Z"] = nb.display(sampleZ, 0, zMax.getRealFloat())
        
        for (int i in 0..sampleA.size() - 1) {
            labeled["A"   + (i + 1)] = nb.display(sampleA[i], 0, aMax.getRealFloat())
            labeled["Tau" + (i + 1)] = nb.display(sampleT[i], tMin.getRealFloat(), tMax.getRealFloat())
            labeled["Pseudocolor" + (i + 1)] = op.transform().hyperSliceView(pseudocolor, lifetimeAxis, i)
        }
        return [labeled]
    }
}

fcd = new FancyDisplay(ij)

FancyDisplay@327d3815

## Loading Dataset

Here we use the [scifio](https://imagej.net/SCIFIO) [bio-formats](https://imagej.net/Bio-Formats) plugin to load time-resolved transient data from `input.sdt`.

In [4]:
sdtPath = "../test_files/input.sdt"

sdt = ij.scifio().datasetIO().open(sdtPath)

[INFO] Reading SDT header


The acquired dataset is actually a 4-dimensional image as we will be shown bellow. It appears purely dark because the notebook by default displays the first layer it sees.<br>
We now use the following snippet to "chop up" the dataset for demonstration. We also display the metadata for reference.

In [5]:
import io.scif.lifesci.SDTFormat

sdtReader = new SDTFormat.Reader()
sdtReader.setContext(ij.getContext())
sdtReader.setSource(sdtPath)
sdtMetadata = sdtReader.getMetadata()

// display the axis type of each dimension
for (d = 0; d < sdt.numDimensions(); d++) {
    printf("Dim #%d: size: %3d, type: %s\n", d, sdt.dimension(d), sdt.axis(d).type())
}

timeBase = sdtMetadata.getTimeBase()
timeBins = sdtMetadata.getTimeBins()

printf("Time base: %6f, number of bins: %d\n", timeBase, timeBins)

cStart = 6
cEnd = 15
tStart = 5
tEnd = 16

table = []
for (c in (cStart..cEnd)) {
    row = table[c - cStart] = [:]
    row.put("Channel", c)
    cFixed = op.transform().hyperSliceView(sdt, 3, c)
    for (t in (tStart..tEnd)) {
        sample = op.transform().hyperSliceView(cFixed, 2, t)
        row.put(String.format("%.1f ns", t * timeBase / timeBins), sample)
    }
}
ij.notebook().display(table)

[INFO] Reading SDT header
Dim #0: size: 128, type: X
Dim #1: size: 128, type: Y
Dim #2: size:  64, type: Lifetime
Dim #3: size:  16, type: Spectra
Time base: 12.500000, number of bins: 64


Channel 1.0 ns 1.2 ns 1.4 ns 1.6 ns 1.8 ns 2.0 ns 2.1 ns 2.3 ns 2.5 ns 2.7 ns 2.9 ns 3.1 ns 6 <img src="data:image/png;charset=utf-8;base64,iVBORw0KGgoAAAANSUhEUgAAAIAAAACACAYAAADDPmHLAAAGDElEQVR42u1dW24jMQzLJXvJXrKLfCzQDpKMJZF62CxgoGhSjy3LelKax+Px+EGOr6+vH/ScGsTxPLAph5a51oMYeZ/NMvZxnXPlGZMu1GPHQ9MwSgCNfkyfeDF0IIdLof6E6XZAmzGMCHi4ZNKBRIjuoV0zeuvQD5d+Z/nxGg4G+E3EaoLqQAmBoOcv39/fW4tKSZcCFaDbKjfwKCb5v5dhe9r3cDvZLqx9APY136/WCNFQ4r/bnpLpohszUZoB59ZhyQsYfJjez8SIQyWAbjaJAWSQvf/+5rTRzdpFCjnXdQahMteiSOBhtxX1jCLGkZEWXXfX4M/inNKth0u0fQ9TDFXEAJMIPyRcy3xGD6Jn5dIt8w+r8TvHCxgKvOgdCFLg5diYQY8CiCh6Z1fgyXWdv8G7oD30IErFs7OZToUh8sFLU9hv5j4LoLn67M4qBcxAMgIVCDr8cCYljODjlNtVbWRuJQHYmSuJ/NSg2VliX8zVxAZQOHdQJLAaLKFijoMDQaotlBdAZZ6oCkN9P3oOgf8/DgatMVUFdC/cGNpI+jxIlld8E5oziAGkIsr3t+chdDz8Co9mQYXOujUZdkDHsDRxHTkPnWIgdUH5JKKX1+DaTL91cjiZie5JAq/MRP94JMqVYe4+76wGS3sEMTj7Fdp1ij3CamubVHSaezOtn99Jk+k1iBU9mk0q4JW4yTYOO7p/78SwRf29+m4BbXnW8OqmGP6x1aCNhJyZnsN/CYGkoVkCZIstj2XNYjbU7WwcmNqn00bEM6hOcllv8KdLuDLfbXHo6gRWl49l2X7adDSo8k5FrO6XiQXw1gqa28S9ujkrFjo6JmBRD8ggzcqhRw46KplCgJCrOOnefnV6Bu9VxW+2O7jcHwANmXr+sDe1ajnvAH71YBX+MOFE0OWdXvV6LoW4vDQ6v5hX5VVXhnoyCysOgrQhQB6NbQFWC9jjpnmSHZ/WxQwhV+UtPqk7I/3qIc3erGJlt7GKjCkpPcwXrRa/na1HPcbSnQtcDROzznGxh2oXk1U0EQlSWQMs0Wd5pZhTQsRELDLQ49mwxw1ajZ4hgj8MqYVYwy8a9LPqPUERVIx/ZZ7VFzl6rP9XeyfbMxxOzUzsWNLQd15G9C2dFalvz9pgnUKrsmuZANZV3bz691W1xTI2IaDQqR3B363bonYyvICySGCBDoISigWl7pJTiKihN0zOE7+ofHlGkMfLOOgDQ6hU415wm0LEvLNDuQxQiTc8jUQTL89fEcO2RAGRxPMYb6veTsWLKKDVwUxAQsQ9ZLlBHWv3ImBYGiCkS1GlV02h1hWpj7Cmcb0ILRczeNEzzCJJb0p4UlVQJDUekaJuQEhGRi/rwC1I2ohhyKipsEidxUsUd7+m1+xZRC4qOGQtG7urbA48oxafliEtmBY6EjqGCnEbJZOaOFWBOSq6rodAodGFeAjwTmcixHSWocnyHlaSVcs2gEfHeIyPuwNAFX8y3EBWFvHOOLwD5gQ9MH6mqqr2gGGceoh99TpYTE8BhKCzg+iEDjrhxIpoAo02aE6lJBJ4V948zRBFFX6wso2psHBr2DIKykR8f7XwwtoBJaqCkhJMmHBuRT/8qM+8cqjVr5tnYC///M8dIrUbjAsdNOmaUVwJFsFrAyuJMTEtvILkASduzBIsDRbOioMjdT8qrYoCiWTmUKDvDUTVALD752R5NIzLwEALw9xARu+dDsCTCCN/SjF7n59kk/HLq6pu9XKbFFJ105CEWN8NdUsVM56FiFIGLy6/T48VxTI5RcysGCbRBoNYYdgHneyMbswJbOJ9JqgjE8xZ3YntxtbJJ152G5jKfj3tDUVke9MdpMiBnVDrkhrdXMsuc7RxA7u/zUM9jQdJAA2sLVP61rBT/fFN1UPfti3TQs9DmUe3fbIqANBCBzptf+BU8ezDy3DdJrwHIPCs2QmSStujkm5QL+Bkt2+XDCTMCOxCDPY6dnsJpOIA8mDOYYDs9qrHhYJPuGliFmNpWHUxh0T/IRJA4r0pA2RY5Yr+SQKU1glIAsgvFgMosXMctlG3QBJA49jxDy/+1tdDW01/AAAAAElFTkSuQmCC" /> <img src="data:image/png;charset=utf-8;base64,iVBORw0KGgoAAAANSUhEUgAAAIAAAACACAYAAADDPmHLAAAMIElEQVR42t1dAXLkRg6bp+3T9mn7s0s5dU4UbTcJgCBb2q1yxbHHM60WmwRBkPp8Pp//RV8/fvxYfh/9DPkd+/X9Xs73rK5l+m+bvmoLzV739XvkNezvpjdy8vOGrw2/ecjC2MWfPBGIcbIe8Q0GcXs//yZ1nOD765Eboax72r1Xb67BOJ7nTrs3JTKmqlF9/f7nz5/vwRSdN19xlxWw5wCk6n6g1/ptHIgRjhxCddO60f8Jdzq13odlAlysVy33AbGOxhEVb9Ad5mxfzgufjHWda+w6zU/gMhafPRtfq1nFG8mfyr4OkG8xWHFe/FS4eCL+uJ7+Cp/SgL3eQ5MimzgZ0qITexrzEH8/i2ArLu0tocMNADuu+/Ke/SzeU2/QFFLv2EfjwZjL8btAoNP1doaUJ4FgyAOccpunPY8DrDoP0dFikLK4jgziREbi9irV92vycu8QNlTSJ8SjnCRpJopBAZFWY8OUMu1pV7gzksx4uskgF81OrnPeujPQdsq7vEHM0pC5zC5UuYDmPLjtRjDpJrKeSGdQuB5/4aWT8nXfuEHK9alElT9te0NdXgV9LKePnmbU8zVUVLGTMBmrvzZnCACdyLuPFdmWf/c27X5H3cIV4jqYPifou3vt/3iAqVTq5U0UkueZSIcLQtc+UuQEkHo7QHP2aID78rwT43TlVbDniOkVmXq1RgJcf69MGrmYExWyk2BxQlgLv7YTBLr5g84eQ4eXmRCWNmQWHCu1a2xw9g+6qmZ/Gr5okqnPU8HK76pudfe+bBsXW13samUzhg08bqtpoLqBT2gmOY3o2f0UmMq5+n2VRlX/phovM4FLRwzvotq3jSFdZEVXPb0CNO9/o4SBk70GEeMohAcf2fAnEzRs99NpiRpxoM9w+0+lh7Ms52mHQfXcNlWwA6i5R8+c9lwoI6mg+xWOKgLm14wysY6uURoynQWfqPYyXOLm0xR2gc6U56lEiyLjUllMc1HofVM31DVd4zt7Yh03Uslw1AyFMCivS1UsuUOR6wwRE+GmU0iS4A9f/p1Z61vmA2RrQzMFlfKekN+ljSGn+gW6O3tdVcHJUnMrSOzg+jsrhE+r9VdPq1qMMjawPkPqtdoIxDhXCuLIeDuN0TlHifUOSPgNNYFq/MqQKkOKVLyLa5iSo9DirIO0h2NHfX4it0eFKm6dgqIAUnN6p7qKWN8sU4eeWNUVTqLpJ08LJw5uLyEzBaaYz+gcYOFQ9FR0iiVN4Imp1Z3ziE5I2DuLZU3e62O1sic92qUK5J50De5eBKg3UMk30T53FCmvcIFbwePaA3eL2O7/VzUNBYQuFUHV04Bsphrj1FiqeLPIGNWMg0mX7/v0fWiqZelF6j6ny6+2oO9AnOuE7sgoJjtxA2ZmbhHqQW+G5NvcHVMXWJ/kJSqDl5EKXMZK7srK6M1X5h+g7j661m0amFk4uskr8mJF26oPVOiQjyHXdD0c0V5lniEzrF+/fkGYp5Im3j/jg5xABJygzNVOlh2dCjcQVQcyrd7/+3XX16+8qPr8JKKur4FNpcS4M5LO5sydIV4BEmuMapEo+u/dY2QhMRN6Xr++D4r5EOAg7G7lUVGIRbxfrmn3mq9/7FQMxOB2+

Shown above are images from channel 6 through 15, time bin 5 through 16. For the rest of the demo, we choose channel 12 and perform the fit from time bin 9 to 20.

## Hyperparameter Setup

Prior to fitting, we set up some fitting parameters specifying how the fitting is done. All the settings are described below. The commented settings are optional and are set to default values.

In [6]:
import flimlib.flimj.FitParams
// import flimlib.flimj.FitFunc
// import flimlib.flimj.NoiseType
// import flimlib.flimj.RestrainType

// create a new fitting parameter set
param = new FitParams()
// the dataset (3D image with coordinates (x, y, t)) we choose channel 12 in this case
param.transMap = op.transform().hyperSliceView(sdt, 3, 12);
// // the iterative fitting routine will stop when chi-squared improvement is less than param.chisq_delta
// param.chisq_delta = 0.0001f
// // the confidence interval when calculating the error axes (95% here)
// param.chisq_percent = 95
// // the routine will also stop when chi-squared < param.chisq_target
// param.chisq_target = 1
// when does the decay start and end?
// param.fitStart = 9
// param.fitEnd = 20
// // the deacy model to use, in this case y(t) = Z + A * e^(-t / TAU)
// param.fitFunc = FitFunc.GCI_MULTIEXP_TAU
// // assume the data noise follows a Poisson distribution
// param.noise = NoiseType.NOISE_GAUSSIAN_FIT
// // the standard deviation at each data point in y
// // NB: if NoiseType.NOISE_GIVEN is used, param.sig should be passed in
// param.sig = null
// // initial Z, A_i and TAU_i (i = 1, 2, ...)
// param.param = [ 0, 0, 0, ... ]
// all three parameters above will be fitted
// param.paramFree = [ true, true, true ]
// // use the default restrain type
// param.restrain = RestrainType.ECF_RESTRAIN_DEFAULT
// the time difference between two consecutive bins (ns)
param.xInc = timeBase / timeBins
// // generates the image of return code
// param.getReturnCodeMap = false
// // generates the image of parameters
// param.getParamMap = true
// // generates the image of fitted data
// param.getFittedMap = false
// // generates the image of residuals
// param.getResidualsMap = false
// // generates the image of chi-squared
// param.getChisqMap = false
// the index of the lifetime axis (from metadata)
param.ltAxis = 2

param

xInc: 0.195313, interval: [-1, -1), intensity threshold: 0.000000, instr: null, noise: NOISE_POISSON_FIT, sig: null, param: null, paramFree: null, restrain: ECF_RESTRAIN_DEFAULT, fitFunc: flimlib.FitFuncNative@7514f5dd, chisq_target: 1.000000, chisq_delta: 0.000100, chisq_percent: 95

All of the fitting ops takes the same parameter, the fitting parameter (`params`) and the Lifetime axis index (`lifetimeAxis`). The rigion of interest (`roi`) is optional (see below).

In [7]:
op.help("flim.fitLMA")

Available operations:
	(FitResults out) =
	flimlib.flimj.DefaultFitRAI$LMASingleFitRAI(
		FitParams in,
		RealMask roi?,
		RandomAccessibleInterval kernel?)

## Performing Image Fitting

Once everything is set up, the fitting routine can be easily started. The op will generate an `FitResults` object with all the per-pixel results assembled into images. Specifically, `resutls.paramMap` will be the image of fitted parameters if `param.getParamMap` is set to `true` (which is by default), and `resutls.fittedMap`, `resutls.residualMap`, `resutls.chisqMap` will be those of fitted data ($\tilde{y}$), residuals ($y-\tilde{y}$) and $\chi^2$ respectively if the corresponding `getXxMap` option is turned on.<br>

This images (`xxMap`) in `results` will be of the same size as the input dataset in X and Y directions. The result attributes (fitted parameters, $\chi^2$, etc.) for that (x, y) coordinate will be layed along the Lifetime axis. E.g. `results.paramMap(x, y, 0)` will be the *Z* (constant term) for the transient at coordinate (x, y), while `results.fittedMap(x, y, 4)` will be the fitted data of the 4th time bin ($\tilde{y}_4$) of the same pixel.

Here we demonstrate the most used ops:

### Initial Parameter Estimation (RLD)

In [8]:
// spin!
rldRslt = op.run("flim.fitRLD", param)

flimlib.flimj.FitResults@690970f

In [9]:
// showing tau from 0.15 to 0.4
nb.display(fcd.tableDisp(rldRslt, 0.15, 0.4))

Z min = -99.99993896484375
Z max = 10.958125114440918
A1 min = 0.0
A1 max = 1728.772216796875
Tau1 min = 0.0
Tau1 max = 5.780399322509766
brightness_max automatically set to 1696.0



Z A1 Tau1 Pseudocolor1 <img src="data:image/png;charset=utf-8;base64,iVBORw0KGgoAAAANSUhEUgAAAIAAAACACAYAAADDPmHLAAAd3klEQVR42u2dx47sRBuGDznnnHPOOXPIOecoISEyIi9IC8QGBEiwY4EQIHasWIG4AXbcAJfBDfSvx/of6+065Z6eGZh2z9RIpWnb5XL5y6nK27Zt2zZpbUu3BoRGAK01AmitEUBrjQBaawTQWiOA1hoBtNYIoLVGAK01AmitEUBrjQBaawTQWiOA1hoBtNYIYJztgAMOaMhsEmC4HX/88XMTi9e3CFGNa0L777//mu895JBD+t+HHnroZI899uh+77333jsg97zzzuv+77vvvpODDjpoK0uTxU7gsMMOm3kdRM7i5lo7++yzeyQfd9xx3e/99ttvstdee0122223Hsk0ni/RcVwjlmyMt8kkxMY/NIFcOw+iTjjhhKlrcDPXZwGd6yeffHL3++ijj56ce+65/fkkpJtvvrkfiz777LPP5KijjlpRdWxSdTEOA65EUkkQngNZcOxKAGesktA4ViI4Vo6DCilVkHNCVUhQSBP+c7wJpMDiHg4C8rdcD1BvueWWKjEowtf7TMY7/PDDp66dddZZ/fPpJ8KPOOKI7r+2gseoD/pyfOaZZw5KtkYA/2/HHHNM/1vgqrPhqpKbOIYDb7311h0402PG0aCTi7nPZ9WQUiK+JvKTOHkW4zA282H8888/v0c+z19S5G+8BBCwBx54YH8Ozk+Aw2kSBMAX2Sn6uQ7Q+Z/3lkSUnkFN9WR/kCuXe16D0nbTTTdN6X2e7fOVDI0ABhoAEmhyGsadxAAneR6dKwLlLpGeauGGG25YU0wAouN+EYhtAAHA2Ryfc845U5KK/ymF6DevemsEMNBKS1+OBLg0dLJWOvpWv14kgqTkwCExPIQMEH766af3aoH50CA0n5Xqgv7MaR53UVXRCCCMpxqwPIa7rrjiik4KpISQKABoiljOgSju0ypfT9h41113nZI85Zh4IimNcpyaTeE13iNtny1DAOli+R/gajypEpI7DbLQIBqALiEoFSCQRCDqo+R8RTXna65gGXwqJQhc7rl0TfVASnf1pJNOmjrHb4hiS0sAxTuiVHEKYEE6iARIAE5DLhHBNZDIMX29HwRoQ5TPS09gKLQ8pCLwMpJza9FJnzkrVK1dYDAq4xdbigASkZdcckmHeLkHJEAcF1xwQScay0hbBmroe+GFF04uu+yybhwIIY3HWUitzWlIVWjwORZ9M+qoV5JSp5Ru/CcWsOeee+4AB4mWMU899dTe6Ny0BFCKPjgDoCUC1bMe81v9m1zI71122WWKywVmchnElufXmi7WI3EckZd2DNeMDaQrqZvKe0LgvI/H/Pd4U0qA3XfffQq5KUa1rlUL6strr712B0NR24EGl3AP13744Ycp8VsC8rTTTptJkKibmkj2eYxtviElTBJg3lPaIsYbgMEzzzzTqQF+q8a0b1CBK+UXlloCiDCBASB4cX7TUmQefPDBM/WpuhQgGuFL5Nfi/UN+umHemu1QupcrSRC9kieeeKJ7DgZf2jGM4bEShfPaMPMkmZaOAIiYmagxYKPoB8giU1FJX67V9LK6E6SpnxOhtQjhELIcq7QdyjyESJ1HZZCn8B0vuuii/r4kJOIXSjwJX6JHFYwoarh+g0/dZyQtr4N4JIJISiNpKFDDOHALgJPD06Ub8gRWQp42BOOAAAlx1hiGqSFGdT/nr7vuusmNN97Yq7oMbdMfdaMaQPxDHBiJxDq0dVQPSy8ByoCICDKSBxCOPfbYDuiPPfbYDrF7o38i6ZRTTun1J8dw2RCxgBR1u3GFWoxfPa0kUIJAnGW0ryRGagsY27gERIFkgkghAIhBAoAoQLjvrXSUcIxqjih59O+5fbp6Ig/AeKzIAxBwANevvvrq3jZIpOAqAaDrr7++DwClNICYhhI7cFrGBDhnUCaBrupZKXycWUvb008/3T8DYkAlSHAGkTjPXAxVS5j5vFley1IQgEaNHGxBBS8O59AggrSYU08aIgXR/DZBQ38QBOJMtTIu40Ec/C4DP/SBK7O8S2+kBmSepToa0se+G/dfeeWV3dx5Z6UVz4Pbcy6MJSNA6BAr9/GOmc1MOCy1BDBSZxbNc/fee+/kjDPO6IADp+OGySEA1FDvVVdd1QEHhBshFEhwXyJPoioNz5ofP9TKAFKKf43LmtUPYrUbMoKYKkRLvzQ+fWYavBqGszyh0RMACICTAAQGDgB67bXXOp2tO6SvDyFwXRcRoMkV6R1YyMm15JJ0CTNcXKabawSa/Rgfwks7gRx/IqysIeDZEl7mDUoPBoLn/egLYRvouvPOOydPPvlkNz+TTqbGDSgtHQFkBQ//L7744t66ReyCYIDw7LPP9oERAANwuRfxa4AERKsz01LXmkY9mAhKncxzV+L4WuUxiCr7lHGClAAal5kkKvt5nrnye+edd+5UBDD58ssvu9+8N3AYqn4qy9NHTQBZOaPoFkhZ9MF/JAUv/uCDD3bABNkQB8B56qmnpgI6BlHUvXAIRABxlBG5oeLStLjnWXvAPczF80QVJTQLRdLINfhjYgvp5zi6wz/99FPXn7mbE9HtRGJy36WXXtrdAxHPKlMbFQGI1IzfZ40c+l73z0JJGjbB7bff3htgcpz+vjpVC5/QMoATuJRyM7bISCSKPIC4FnGaqWsQ5flSP5tQQq1J8Lx3ejs///xz904QPDCQUfRohqTSAnMEay/jBpFOPPP1+MIaceplEAsRgMSUHohMLXijh/r13AOQGQ8gixDHpB/cw//XX3+9d79qqVt0b7k8TO4bKl7JoBEI530gXOMaSC/mxXXnToNAdHOTQCDQt956qw9zSxDMZaeddqo+f3QEoIGk8QKH8jJyeRprII5jbQNfGIQpWgEkAMUmgPtqHJyITOnDfejzMppmybcGVy3Ik6pCbtdNS+OUZ8DREt/27du754EoEQ2CiQUwvjEPfluv+Oijj3Zw4L2NGaTUwQj1nlFLADnIqJ5+vrrN9Xj047cAwgZQTQBM7ucejjMHL7KtywMwpYhMAqnl1JVMSBa421Qu9ytZsiqJeUJ8zFFOznj+m2++2Qd0mDdEiqp69dVX+7pFVxhpBPIcmIFrzJHj2267rYML99DXGMIIqolXH/xJjlZkASTFocEP+1PUAZAAssQAoFKUw7FGzAwIYQeUz3/88cf77KLzKPtdc8011bLwNDRLj6a2dkA3NyUXSGXuSAka7h1ur0QvMXPN3MiRRx7ZjWNK2OinQTJtBhlgpfWSCyOALHxIHcbvE088sXthjDUQoGjn5QAa+hDAKUprKWJyAAJxKEACEhmvrNfPXL/ZNrgMwDPfXFEE4HHTSsNL15N79NM9D0Ej0pkvyP3ll1+6iuI77rijJyhFvp4Az6EP8xBWeg1EFhnHELfEvICS8tXp/9oiCBdUgBgjg7zYO++804k7qZsXlgvgmlQBnJNoMnVcW9tHXxHKM0uOQUwzH+aBGjIJpMRyESnXGU+9nZFAOJPfRDB5luHghx9+uCNyDTkIm8a7pSrIghfu5x6eQ1yEZ3EtCYc+ps43ODA0/zJuuZIJktZMxEAEcqUGIpzEPVrHZfGmVT8GfLgPrkkRXhpxGpwZ2ctiUa47T5I2jM85c/N33XVXH7WECHkeiGUMI5i1fQQkesbCnX377be7vkT5eI5+vLGANExdf8A1kM57gnSIxXfSLhqtDZDcn2vwUp/ykvrtAB6Rb0XQSptAGCVMUW3gx2cwFoA24aShJne7SJN+SAYikIhfEzkcOxfmpiGmJU9wirGxM+hzzz339Ig09KxaSyLmHLaAno0WvoEgiMV+zmNo15J5Vj4vhAAy5MnErcETGby8lr2iWaR4LhMxaTyWkqQWdQSpIiYLS4waiiDEsUSHy6UtYKJIBH7xxRd9sSb+vNlGiMC

### Refinement (Levenberg-Marquardt Algorithm)

#### Plaint LMA fit

In [10]:
lmaRslt = op.run("flim.fitLMA", param)

// showing tau from 0.13 to 0.25
nb.display(fcd.tableDisp(lmaRslt, 0.13, 0.25))

Z min = -0.3008432686328888
Z max = 5.321786880493164
A1 min = 0.0
A1 max = 1088.684814453125
Tau1 min = 0.0
Tau1 max = 3.395394802093506
brightness_max automatically set to 1696.0



Z A1 Tau1 Pseudocolor1 <img src="data:image/png;charset=utf-8;base64,iVBORw0KGgoAAAANSUhEUgAAAIAAAACACAYAAADDPmHLAAAuiElEQVR42u3d1Y5sWxXG8Y27u7u7Owc4B3d3d3d3d3e3C1yDhZBwAc/ACxAueIoivxn+m0Wdqu6q7qre3ftUJyuresmU8X1jzDHHlHXq1KlTs6M8LnrRi24knQtd6EInrszH9Di5hZ8Cc8lLXvL/7l32spfdaF7z6R/FceELX/iCRYC9tHrTgO6OY0CAozTju2MLBDgiE7UjzVEQYK827qBO0MUvfvH/+70uYabvLzsucYlL7Jy8o7QAR+kMrZLXRS5ykRPn+R8zwh0/H2ATWn02gHPiegGbKvDFLnaxjZZjFUuySWGfiS7jVgmwn+O1Thftcpe73LF0ADel+SfQn9gO0y91qUut1W7PO3yLBJllmDYR85pbOvNkWUbSC7gDuJkmYD8hrmMStw3IXmXZr5xnaRf0cO3XKl21bXnq6xy7SOIeBFgV9P20c9MaUrmueMUr7ukc7uXATeum/PPNx6Uvfel9y79fvQ9jta50pSudfd1AQp/vyp2JtlY5gArorNC8b7LIOi0jwmUuc5mzxfPfLgHmwd5kFyvApsBNActaBPQyq4Gg0nCuvEdJ0mPkfB5NJYAUOKv4EFPTDCht+KL361pe/vKXH+W56lWvepoIV77ylYfW+i2fReVFAHnRYOkwyfKLOMrdb+9P0zgIiMfQUhyPiFv/T0EGWk1JbWXgEOR1rnOd2RWucIUBdOAA/FrXutZ4jxVwnwWSFoAR5LrXve54zjOuX+Ma1xhpe9Y7yCSP8vHslBSeOYt6BJuN2mXunRdpyPy16SARQG5wgxsMoBCBoOebj6tc5SpDY2uPEYN1UA5puQ9QgEkDUAhx85vffFx3TR7Xu971ThPMM4iBAJ5BKL+9k//giDjyz6rs5ReckNHO85ulaft6kC7cokosMn2uAe5qV7vaEKbfAPA/EByAIWRa6Z6zA+iIAkC/ne9whzsM0lz96lef3fSmN53d9ra3He/e5z73md3oRjca5LjZzW42SHPNa15zpA9klsT/8peWfAAvb/9XNvlJH4Fc944062aeieHxE9kETE07IRMoUCJC7T0B01TPpG2eu9Od7jRIQODeyVTf+973nt3whjec3fKWt5zd8Y53nJ133nnj2Yc//OGzu971rrOHPvSh48wi0HAa7bj97W8/8kIOhEAgwErT75vc5CaDQAjjfXnL1/OuI0YkOSsJsMgKLDN9i7R/+j7QCNI14AKaoF0DHGIAHNDuAwaofrsGvLQUEMyxAwg3vvGNhxVwftjDHja7//3vP9J/0IMeNAC9xS1uMbv2ta89O/fccwd4nrvzne888gW8Zz0jT+khpnvOt7rVrUb5lQPQ0rn+9a8/3kGCrMYyx3AT5NiSA7nZKJmK7tVsuMfkAtwZoLTJmXABRescgPe8e4RN6FMtpb0RLmKUJq31rPcCFRH8pv3Svte97jXueyeyIYR8gO6QhrLKi/Z7l8WQzt3vfvdBIiTQ5Cg7Uh5WpkfcRVwvhHqQdo5W115nJQgekAR8m9vcZmiQNpsQHbe+9a1PE4XwaaBnCBphAAwMph5JpO8997IK3mNNXAO4Z4BMwJoGQKsP0JThbne72/iNDMB3X573uMc9RtqsSwTM2fTeox71qFE+70nbdb7COvMaDjNGsXECTBm4qle6jLUJAZGA4X+A0CLNyF3ucpdhZmkRwAke6Myq/wHnIGBaR/M1BTQPSbwPFOX0jmtAAYZ0chaBwjJ4Rh7abIKVFiJ5Brnue9/7jneRB1mQAlnkD/jIJx1NjPzURRmk6R0kUteAW2T+91OkI3Qot9NFmfoIyEEj00ZAEiAyENI973nPoV3+BwaAAEzgDoB6HthIIW3aSfs5fpw81+q75+TJD3kiH3AQjfZ6Hsjy8Rt4LJB8ACx95JBnbb605CmtabOEwECv++o9z7hfTEI99wP1DAWJDq7dexV4WlnCZRJpFgEClOb4n3kFMiHSPtqkHaaNtCrt195qg73jGuECj6fv//r4gQ00JPA/oJ0Bj0C3u93tRjrK4X95IyFTLn/vOyuXvPM9siKe9z9Se0YdWSb5y9d15fdewaVjMOizGgEIai8nZB0HBfAqTrtpSOaWUGk5Qdeu5vDlmQMpZ4sJdt07fgPSfdp6v/vdb4BJywg/J0xa8n/sYx87wHCt2ALyyY/GIyDgpOGQz0Me8pARO6iJ8b5yeF7+LISy1GuQJnI4lAlJWBvkYhXyfw4zLL61LuZBI097mTMgSBfghEIDgUGghEdAjhwmZtx1gBN2WgkMgkSUBzzgAae1sXEFwubJswwswtTUNz5A+1wjfKZe+g5pa45oLlJq670nVuBdZSro5NDMsELy8U4+A/DdVz/XpalsnpMvC4Fs7hWeXmf21NaPTXU55kfVAEpTXQc2zaFNQAYczaNRAGQF3E+b0x7XEcE1VgFIwK4NJ2D5PPjBDx5AsAp57w5p5NnXNufAVW55ISYSABQhI5S0lBexXNNE1XQBTRPoDOSIXVdUutJCWuX1/7I5kcsGuY51JDBzNj86poJFyepLA9mZEIGu7SWYpz3taUNwDlYi7QE8jQGqgM6TnvSkAQgwpUuYwCwuL98cLv8XywdQ4VzlAnxRxmmTVI+EptbUABqwyuzMR8ip9J40cv4av8jCsR7KI+BUr0HZ5KGse/UOjhUBps3DfJ92OujTOYeLqfRbNwmYHD/aQ2A0n8Cf8pSnjGueBzZh1AYTFM3yHMeMID2PPCwBgFkTgiR8wqV1zHAmmdYVXkZWFqGeh2c8C0x5Aa5hY+VhCZBM+y5dJEC8gk/S8k5xh4JanmPFIiviq1NdRuRUB+97t1HKYx0K3o+lsTl2ExoB1h4Ch0ak/Xn7BMPsIwghELDftKUmAHFcI0BnQncUsaORCEWL+QjKkjnN1DYMDCT3kKHegbrJR7nrKShfQSNld73YQQRyXxke+chHjms1U64rBxKwWkiqPv5HbGX1vDTlu5dsj2xUcJEzN++tTqdUzU/ayLslLG0wAgCrGL4zTWbyaQVBRhJtd+aUVgCA911kkIAB7f+XvvSlgxCZ7kbigFfvAQmBC3Rlq/+OiPJWBpbFtXo8COVdILpf8wBIoCqH+4WP5avsylba6svq0XJkf8tb3jJ6ErqodRnVq3By/sGqIG85KHT+rt0qbVPmKy/ZNQIk0GL0hA401wiIcAhM5QugSKc+MgCBIj2CYiGK3xfPp/VpZ05WPoF8AeX9mgT3iulXVvUroNOYQcPQQHZ2NOgkzbqRrstHswRg5h4BlO+cc84Z9ZSmaw7AI5LnCiOzDKyh9Dat6Wsvqzto9KkxAuQhaFr+6Ec/elgBgDsTEPP3uMc9bggiUAlj2md20LymZvkfyI9//OPHe4iUyS8QQwt7fzpr2G9CzlP3jDRz2ADvOmJ4V9mlhSTIVt51J5sx5H5dVOCpo+vaeaADk4n323NAVm7WAyHqesrf++RDIc743MAKsC4Tm0OXGSQsAAMdCMy2Cj7wgQ+cPeEJTxgaSksaaKFFDfR4n4D8RgJComFAo0W0vJ4EK+E9RAISEPgbfjv8BmLdq4I8tJgn756z/FyXNjCUockntAhxyKZuZyN/DQ/nnAJTGsBnPYDtGYSlFOTA50FkaTzxiU8cSqFcnieTqRI2p3Evv6Am4aDWI8fzv0p8ai1PtJi2g1AJqUEbIBEmIAGHAA4Cf+pTnzqEqm3Nk/d/ET/v0lBnvsEznvGMARZN41A1RuDdJo1kAeTvf2REDKRAuOYPACSNrpmQbkPFpcOCALDhZYBLu2atHo1DPsrO2jTjyDMILz/1AH4hbHmxCsB33TPKV7fyDFqC9capY12TMAg2AhAQoAHo/8c85jFDAAhAw1Q6QSAGR+mtb33ruM+BShu9T2gEDXDCJtD

The plaint LMA fit starts with an arbitrary set of initial parameters $z=0, a_i=1, \tau_i=1$. By design, the algorithm is only able to find values that locally minimizes the residuals and is therefore harder to converge compared to the following scheme:

#### LMA fit with estimated initial values

To set the initial parameters, either use `param.param` to set an array of global initial values:

In [11]:
// Z = 0, A = 1000, Tau = 0.187
param.param = [ 0, 1000, 0.18723493814468384 ]
lmaRslt = op.run("flim.fitLMA", param)
// later fits shouldn't be affected
param.param = null

// showing tau from 0.13 to 0.25
nb.display(fcd.tableDisp(lmaRslt, 0.13, 0.25))

Z min = -137.22894287109375
Z max = 459.2732238769531
A1 min = -456.62646484375
A1 max = 1088.8184814453125
Tau1 min = -1.5415652646912E13
Tau1 max = 7.7091735513491046E17
brightness_max automatically set to 1696.0



Z A1 Tau1 Pseudocolor1 <img src="data:image/png;charset=utf-8;base64,iVBORw0KGgoAAAANSUhEUgAAAIAAAACACAYAAADDPmHLAAAESklEQVR42u2d62oDIRBGkzxMfub9366lhcCyrPrNeFlHT+Gj0CYlrse5OdrH4/H4mVnv9/tn9s8YXDwEAEAAgAAAAQACADSdPp8PACAAQACAAAABAAIABAAIABAAIABAAIAAAAEAAgDkBYDmSywAAgAEAAgAEABwkgcAEAAgAEAA0E3P55MJ2BUA6+QDCy4A7QxAJAsQzFqxCrAACAAQABD1AwABEwBMBgCABAXgO3F/33ecxEBjbj/w4+BzDyLKQ0p9TvXzq89jGQvwhaDl6o9sRSa3guNWz+QrYaglnGj8fVZ+jWtYyfcHGOe41d/aLYx+0CmwA5v/egDO0f7owd71cI9jDW7N+k3Aimb+ypIp49wmCCwBEb0ucAZgAcgpjHjG8nq9/rWABVwvIlYmxbN6U6veUyhaMg2cdS+h5K+tMLRyAZO4kPWLL6UJqy31Wlf7ZK6hf7DUygSq0bYnNy9ZiF6Z0DIW4OqBth7c9+/9fbV8oEqV0mI9aAqdNF1LrfzcaxYp9qwdBJ59/TE9S/2sZ0AWDJq4+++pFav65NR7aycuWDNM7I6ZKwvgDVQtAASO+scXgkbtAn6rcxaLkdu1VDOH4K1v81sAdSJzuX8q0PvGB1fxgrdaCACNagRKbp6b8JwVylUHj689WpSA/X739QTW7BOoUXTOf1/BcF7ZVxOtbvem4g4sQKU/tFT7UimgCoDa6KFaKgAwQGCpqpUm67ySS1DkNomUJta7u6BCuQBLbb4m8LLo+N4jHKVJt5Z9tywEpYKvnk2jpSg/Fd1bYVmwI3jMuYDWhZVcMHY1yYqPLwV+atwQsC7QN72znBWw5PJqRa8UI1gtkJJGBnMJ44o+JR9qKeKc33e18q+i/hyIqR6/UsoXPBAcZ/rVoCo30Wre7yn7elNDtWawhQuoMZdK2Vat9ilWI1fEsa7uwH0D99f8SxOk/K72ta3avgK6g/uzAO+EWZo+S1bDu82slKC3LQVb82Krj08Fcmp10Wp5Ah8AHd8U2gOW3Kq0xA6eAhXbwYMbRjytXaVAT+3+9QSu7AVU3hji7SRSOnqtcULp9aVYhM0gseDjOX6l7gx68ny1tlB7OdZWPYE1BzU8t41ZS845K5HaPFrs5PN87eDWjiBPLOFxCYucBJoTAGuLVW0LtzVobOHWAKCRC1EaPVvsXlr2DQCg8/l+S9BmaT5VKovEAJNd9GCp51tdyMK9f/EAsG7wqOcKPRs7nA6+uWFEPQjiubWsdNnTYj4/RhBobejwHvlWdhlXvUMhRBbQuoVcPSbOBRFBbgeznNrxnP7ZQPEm3LP6lfhi4VtAYgNQcwavtNL5n0TBLECL9vNF9/W5JQwBAAIABAAIAO7ocAaABQHYIDtgwnEBCAAQACAAQACAAAABAAIABAAIABAAIABAAIAAAAEAAgAEAAgAEAAgAEAAgAAAAQACAAQACAAQACAAQFH0C90v1OB35/s1AAAAAElFTkSuQmCC" /> <img src="data:image/png;charset=utf-8;base64,iVBORw0KGgoAAAANSUhEUgAAAIAAAACACAYAAADDPmHLAAA2oUlEQVR42u3ddbQ1VRkG8IsFBiIKYgc2ogJ2B2GjGGAHIDZ2B4IJomBiYmErYoIBCioKqGA3di/1L/4e12+v9dw133wzZ+Kcc+8Fv7vWrHPunIm93+d5n/fdMXtWVlZWqqHbVlttVY05/mIXu9hm+y5+8Yu3Xiv/992jfs36sUPK1jymWT6/t12nec8p9xpy/NhzFrRNI0AXgFOAGWqUrmvVj5tFpKFlaatb37ldx0whwiJsN4kAQyo+xkOHGGsWePXPXHeIkRdtuDGgKOesY9YB4PkVYGqh+0Brk/OxXtpFuDHlXITnLnNbcllWFharxu4fet4YeR0SJhYJxKLAWUfC9RegT9a7vLcJ4Nj43JXwNVWk7bptZe5L8rru0Ueeqb9tkG05jB+aJM4KDfMqxxiyzgotY8JTH9EXFbbWLQQss+BTDTcPOZr3HKp282xpCm+QsLA2ycqQNnjfb5e4xCUGq1TdE4e2TOZVk7EhbkMkn4vo1Bgj92M8s7kPAZrXroPb9KyuHGQM6cYSZRHevaZNxSk3GNNr1xfvmvE+SV2bt8e49XOGeJ/j6z2QdeLkfn2dTl0EXSZAaxYCpjCuzXvG9ga2HTM0zrcZv37uda5znc3KOCvB7OrAaarMRXAblrlOkf++/v+16kTpql9fWJj1+7x2mprcrqsCtBW0baBkiCyPGezpiqtd+UCzbyC/CSu2+nGuXQ8DXfeelbAtK8lck9DQVfGhSU1Xu7kttk81UFdczzWV0dasR8AN8FtvvXV1qUtdarP6NcvWFSamJLdDE7x16zAaKv9jEp6mRzXP7evhmxWjA2zzepe85CXL5zbbbLN6TDaghwiO82m/7/l0DoL4zP9DO42WmeStATGmF7yuDkP67euZfV/7PdeOhwasZhaf5mFA9rnjjjuW47fddtvqspe9bAEWMWyXvvSly+a7/c69zGUuU47N7/b7dK2Qpk6oev1nqecsTx/rdEsjwpjOm64seohHd6lEzo2RGRwQjO//K1zhCuU7QHIcgAAbcHfYYYdq5513rnbaaafq2te+dnXDG96wuu51r1td7WpXq65ylauUFsH1rne96lrXulb59P8NbnCD8nn1q1+9uuIVr1i2q171qtU1rnGNcp5rXelKVyrkcB/lCDHsi+KsVdxeohLM1zPV1RSre3FXklePzzGyzXeA+O7T/0jhnMtd7nIFAOBsv/32q593uMMdCugAvulNb1rtuuuu1d3vfvfqzne+c3XXu961fPd53/vet7r3ve9d/vd5y1vespDgZje7WXWLW9yiuv3tb1/tsssu1U1ucpNCEMSy+Y4gvt/4xjcuRFO2hI2hHUAbcIBoZWYzaUpi03ZevRcvBrOP9zL+Na95zWq77bYr/wMVgLwUoH4DPCIA/8pXvnIx/G1ve9vi4ST/dre7XfXgBz+47HOu7y95yUuqxz72sdVTnvKU6uEPf3j1+Mc/vnw+5znPqQ466KByDhI88IEPrHbfffdCHL8jwT3ucY+yb++99y7EueMd71iuiyT77LNPtcceexTyUJsQdOrMoXnnLiwkBMwz3Dm2QpF5QF//+tcvRiW9vOtWt7pVAeAud7lL2X+nO92peLbffSIFw9/oRjcqCsEzyTLJftjDHlYdcsgh1dOf/vTqSU96UvWsZz2reulLX1qIANjnPve5ZeP9jnnc4x5XCPCEJzyhutvd7lbIAvC99tqrlOGhD31o9chHPrKc6zjH7LvvvtWDHvSg8v1+97tf+UQ6qtAXFqbE82XObhoUAmY1+4Z2rtQ3YJFsMgtkhubRvBzgvI6cx/Oog2OBfpvb3KYAwTtvfetbFxUQ1xGBcjjmmc98ZnXEEUdUr3/966vXvOY1hRSPeMQjyjnPe97zivc/9alPLUSxnxI86lGPqvbcc8+yIQUS+u56QEYWKqKs9rnmQx7ykOr+979/UZX73Oc+1f7771/IrByzWg1T8qWx0+1Gh4BZffZdAHeFijZipOKSOQYCKHAZF6jk1P9PfvKTCyg88NGPfnT1gAc8oDrggAOKB/JG5OCBJJvHkf7LX/7yxfMoAwlHKuDwfGTgwb4D68UvfnG5tvu7LyCBh3gSP2QLoK4jjFAH5VIOZbjnPe9ZyrPffvsVIimn/12fWimHOjbHMsYMO0+Zhby0ZuCsSY5Dpd81NLN4P0+XmAERGPF0hrWP9/s8+OCDC3g8nWwDw29ygnvd617lGghBfu0THoQQIMoXqAOSOBYxbK5Fvl1f/KceNuCL8UB0T4RQJmTj+ZTkFa94RSGUsIIIyoLA

or use `param.paramMap` to set the per-pixel initial values from a previous fit:

In [12]:
// here we use the RLD's output as our estimation
param.paramMap = rldRslt.paramMap
lmaRslt = op.run("flim.fitLMA", param)
// later fits shouldn't be affected
param.paramMap = null

// showing tau from 0.13 to 0.25
nb.display(fcd.tableDisp(lmaRslt, 0.13, 0.25))

Z min = -0.3008432686328888
Z max = 5.321786880493164
A1 min = 0.0
A1 max = 1088.684814453125
Tau1 min = 0.0
Tau1 max = 3.395394802093506
brightness_max automatically set to 1696.0



Z A1 Tau1 Pseudocolor1 <img src="data:image/png;charset=utf-8;base64,iVBORw0KGgoAAAANSUhEUgAAAIAAAACACAYAAADDPmHLAAAuiElEQVR42u3d1Y5sWxXG8Y27u7u7Owc4B3d3d3d3d3e3C1yDhZBwAc/ACxAueIoivxn+m0Wdqu6q7qre3ftUJyuresmU8X1jzDHHlHXq1KlTs6M8LnrRi24knQtd6EInrszH9Di5hZ8Cc8lLXvL/7l32spfdaF7z6R/FceELX/iCRYC9tHrTgO6OY0CAozTju2MLBDgiE7UjzVEQYK827qBO0MUvfvH/+70uYabvLzsucYlL7Jy8o7QAR+kMrZLXRS5ykRPn+R8zwh0/H2ATWn02gHPiegGbKvDFLnaxjZZjFUuySWGfiS7jVgmwn+O1Thftcpe73LF0ADel+SfQn9gO0y91qUut1W7PO3yLBJllmDYR85pbOvNkWUbSC7gDuJkmYD8hrmMStw3IXmXZr5xnaRf0cO3XKl21bXnq6xy7SOIeBFgV9P20c9MaUrmueMUr7ukc7uXATeum/PPNx6Uvfel9y79fvQ9jta50pSudfd1AQp/vyp2JtlY5gArorNC8b7LIOi0jwmUuc5mzxfPfLgHmwd5kFyvApsBNActaBPQyq4Gg0nCuvEdJ0mPkfB5NJYAUOKv4EFPTDCht+KL361pe/vKXH+W56lWvepoIV77ylYfW+i2fReVFAHnRYOkwyfKLOMrdb+9P0zgIiMfQUhyPiFv/T0EGWk1JbWXgEOR1rnOd2RWucIUBdOAA/FrXutZ4jxVwnwWSFoAR5LrXve54zjOuX+Ma1xhpe9Y7yCSP8vHslBSeOYt6BJuN2mXunRdpyPy16SARQG5wgxsMoBCBoOebj6tc5SpDY2uPEYN1UA5puQ9QgEkDUAhx85vffFx3TR7Xu971ThPMM4iBAJ5BKL+9k//giDjyz6rs5ReckNHO85ulaft6kC7cokosMn2uAe5qV7vaEKbfAPA/EByAIWRa6Z6zA+iIAkC/ne9whzsM0lz96lef3fSmN53d9ra3He/e5z73md3oRjca5LjZzW42SHPNa15zpA9klsT/8peWfAAvb/9XNvlJH4Fc944062aeieHxE9kETE07IRMoUCJC7T0B01TPpG2eu9Od7jRIQODeyVTf+973nt3whjec3fKWt5zd8Y53nJ133nnj2Yc//OGzu971rrOHPvSh48wi0HAa7bj97W8/8kIOhEAgwErT75vc5CaDQAjjfXnL1/OuI0YkOSsJsMgKLDN9i7R/+j7QCNI14AKaoF0DHGIAHNDuAwaofrsGvLQUEMyxAwg3vvGNhxVwftjDHja7//3vP9J/0IMeNAC9xS1uMbv2ta89O/fccwd4nrvzne888gW8Zz0jT+khpnvOt7rVrUb5lQPQ0rn+9a8/3kGCrMYyx3AT5NiSA7nZKJmK7tVsuMfkAtwZoLTJmXABRescgPe8e4RN6FMtpb0RLmKUJq31rPcCFRH8pv3Svte97jXueyeyIYR8gO6QhrLKi/Z7l8WQzt3vfvdBIiTQ5Cg7Uh5WpkfcRVwvhHqQdo5W115nJQgekAR8m9vcZmiQNpsQHbe+9a1PE4XwaaBnCBphAAwMph5JpO8997IK3mNNXAO4Z4BMwJoGQKsP0JThbne72/iNDMB3X573uMc9RtqsSwTM2fTeox71qFE+70nbdb7COvMaDjNGsXECTBm4qle6jLUJAZGA4X+A0CLNyF3ucpdhZmkRwAke6Myq/wHnIGBaR/M1BTQPSbwPFOX0jmtAAYZ0chaBwjJ4Rh7abIKVFiJ5Brnue9/7jneRB1mQAlnkD/jIJx1NjPzURRmk6R0kUteAW2T+91OkI3Qot9NFmfoIyEEj00ZAEiAyENI973nPoV3+BwaAAEzgDoB6HthIIW3aSfs5fpw81+q75+TJD3kiH3AQjfZ6Hsjy8Rt4LJB8ACx95JBnbb605CmtabOEwECv++o9z7hfTEI99wP1DAWJDq7dexV4WlnCZRJpFgEClOb4n3kFMiHSPtqkHaaNtCrt195qg73jGuECj6fv//r4gQ00JPA/oJ0Bj0C3u93tRjrK4X95IyFTLn/vOyuXvPM9siKe9z9Se0YdWSb5y9d15fdewaVjMOizGgEIai8nZB0HBfAqTrtpSOaWUGk5Qdeu5vDlmQMpZ4sJdt07fgPSfdp6v/vdb4BJywg/J0xa8n/sYx87wHCt2ALyyY/GIyDgpOGQz0Me8pARO6iJ8b5yeF7+LISy1GuQJnI4lAlJWBvkYhXyfw4zLL61LuZBI097mTMgSBfghEIDgUGghEdAjhwmZtx1gBN2WgkMgkSUBzzgAae1sXEFwubJswwswtTUNz5A+1wjfKZe+g5pa45oLlJq670nVuBdZSro5NDMsELy8U4+A/DdVz/XpalsnpMvC4Fs7hWeXmf21NaPTXU55kfVAEpTXQc2zaFNQAYczaNRAGQF3E+b0x7XEcE1VgFIwK4NJ2D5PPjBDx5AsAp57w5p5NnXNufAVW55ISYSABQhI5S0lBexXNNE1XQBTRPoDOSIXVdUutJCWuX1/7I5kcsGuY51JDBzNj86poJFyepLA9mZEIGu7SWYpz3taUNwDlYi7QE8jQGqgM6TnvSkAQgwpUuYwCwuL98cLv8XywdQ4VzlAnxRxmmTVI+EptbUABqwyuzMR8ip9J40cv4av8jCsR7KI+BUr0HZ5KGse/UOjhUBps3DfJ92OujTOYeLqfRbNwmYHD/aQ2A0n8Cf8pSnjGueBzZh1AYTFM3yHMeMID2PPCwBgFkTgiR8wqV1zHAmmdYVXkZWFqGeh2c8C0x5Aa5hY+VhCZBM+y5dJEC8gk/S8k5xh4JanmPFIiviq1NdRuRUB+97t1HKYx0K3o+lsTl2ExoB1h4Ch0ak/Xn7BMPsIwghELDftKUmAHFcI0BnQncUsaORCEWL+QjKkjnN1DYMDCT3kKHegbrJR7nrKShfQSNld73YQQRyXxke+chHjms1U64rBxKwWkiqPv5HbGX1vDTlu5dsj2xUcJEzN++tTqdUzU/ayLslLG0wAgCrGL4zTWbyaQVBRhJtd+aUVgCA911kkIAB7f+XvvSlgxCZ7kbigFfvAQmBC3Rlq/+OiPJWBpbFtXo8COVdILpf8wBIoCqH+4WP5avsylba6svq0XJkf8tb3jJ6ErqodRnVq3By/sGqIG85KHT+rt0qbVPmKy/ZNQIk0GL0hA401wiIcAhM5QugSKc+MgCBIj2CYiGK3xfPp/VpZ05WPoF8AeX9mgT3iulXVvUroNOYQcPQQHZ2NOgkzbqRrstHswRg5h4BlO+cc84Z9ZSmaw7AI5LnCiOzDKyh9Dat6Wsvqzto9KkxAuQhaFr+6Ec/elgBgDsTEPP3uMc9bggiUAlj2md20LymZvkfyI9//OPHe4iUyS8QQwt7fzpr2G9CzlP3jDRz2ADvOmJ4V9mlhSTIVt51J5sx5H5dVOCpo+vaeaADk4n323NAVm7WAyHqesrf++RDIc743MAKsC4Tm0OXGSQsAAMdCMy2Cj7wgQ+cPeEJTxgaSksaaKFFDfR4n4D8RgJComFAo0W0vJ4EK+E9RAISEPgbfjv8BmLdq4I8tJgn756z/FyXNjCUockntAhxyKZuZyN/DQ/nnAJTGsBnPYDtGYSlFOTA50FkaTzxiU8cSqFcnieTqRI2p3Evv6Am4aDWI8fzv0p8ai1PtJi2g1AJqUEbIBEmIAGHAA4Cf+pTnzqEqm3Nk/d/ET/v0lBnvsEznvGMARZN41A1RuDdJo1kAeTvf2REDKRAuOYPACSNrpmQbkPFpcOCALDhZYBLu2atHo1DPsrO2jTjyDMILz/1AH4hbHmxCsB33TPKV7fyDFqC9capY12TMAg2AhAQoAHo/8c85jFDAAhAw1Q6QSAGR+mtb33ruM+BShu9T2gEDXDCJtD

_Note: if both initial value settings are present, the global values will be overriden by the pixel-specific values._

### Global Analysis

In [13]:
globalRslt = op.run("flim.fitGlobal", param)

// showing tau from 0.13 to 0.25
nb.display(fcd.tableDisp(globalRslt, 0.13, 0.25))

Z min = -4.0
Z max = 5.352066993713379
A1 min = 0.0
A1 max = 1058.4622802734375
Tau1 min = 0.0
Tau1 max = 0.18723493814468384
brightness_max automatically set to 1696.0



Z A1 Tau1 Pseudocolor1 <img src="data:image/png;charset=utf-8;base64,iVBORw0KGgoAAAANSUhEUgAAAIAAAACACAYAAADDPmHLAAA3GElEQVR42u3dd5AtVdUF8EFQETNmQVFRTJgDCpiVoIgiBkQxYMCcMAAKBswBMGDAgAkBlSBGUEwg5oA5JyQYS6mCf/ur36la8/Xr1923+4aZefqm6tade2/36XP2XmvtfUKfXlpaWqqGvjbZZJNqzPGXucxlOr9rlpXPk65RL7N+7JC6NevTLCuvSTaY5lpDbDvWvnN6jQdAW0W7nDHWSX1G6Tq/ftzQOo4BOmdOC4xp2jztuTMDoI2VXUydBflNh3U5r64W+b+t/LF1HPsa45RJYFkFB8+uANNW+rKXvWwvG9vkfCxLuwA3RoLnwdxFvhZcl6WpZXko88Z+3waKtmsOddzQ+D4pFi8SKKsIuMmNm2S8LjY2zx8bn7sSvqaKtJXbFSqGyvMY8Ez72xp5LV4a60aclCzVQ8NQhZjGIZPObQPT0Ox/6DXWCFhml6ZpQTHWObPG9a4Mf5YewjSvTTfddC2FhZXJkLsM3/Z7129XuMIVBvUAmkoztGcyq5qMDXFrIvkc0n8dK8V9sXvoGEFbvS53ucutd079Wk1m1QEwqUs7VBkmAUUdZlXNFe0qTlPZIV22OKePfW3xPs7abLPNOqWzfs4Q9jk+jskr10kdJw06DQHoBtb9a1eAaWLhpC5aX3bd/Nw30NN2fF9srf9uTKIvwewbwBmbJ2xgr6WJXbRZ1aFvFG+e7BjSjZsEoma9p52zmIXNKwy06RO5SRMq0xirLX8Is/vUKjLf1hUL84UVrzogndMXAuqhY2yvZVFdv7kCpKvhTeOPGQRqG7sfMqgzZmy9mV/U65k2xdnk3/tVrnKV9Yan64BoljUmaZwldK5qeBkq/2OHUrsmfLrA0MfuZrLWzLR9r4fg/fKXv/w6bHes37xzvv+9139PeTl38803bx3Qmfek0wYzGTQ0KZs0LVrP7Ltkt+7kOkPjrLbM3W8BAOfe+c53Lu9XvvKVy9iB3zh1iy22WOfle+de8YpXLMc6Bgi8nOddOQFMvTfR1uUbApQx4XGW3Gwu3cC2hrR1waaZ1Wt2E2NchuYcxvf56le/+rJDAgyfOStOuuY1r1nd+MY3rq5//etXN7zhDcv/N73pTasb3OAG1XWve93qJje5SXXb2962/LbttttW22yzTfn9Rje6UXk5/9rXvnZ1netcp/zu3Xm+A44rXelK69TD56jFSrF9gWox28hU1wBKPSnrQnSYjWHiMyN7BwD/M/S1rnWt8u475/mfE7beeuviPMd57bTTTsWxN7/5zatb3epWRQV23nnn6n73u1+14447Vve6172qXXbZpXrIQx5SPfCBD6zufe97l/eHPvSh1R3veMfqlre8ZXX729++utOd7lT+dz4AAQSwAJW6+N/xW221VbXlllsuD04NHd5dg5NH46Zbh0zbtiVuHF0/LmzHsjvc4Q6FpRx5jWtcoxh6u+22q252s5sVwzM2JnK8dwylAM7zGwZz8q677lrd9a53rW5zm9tUe+65Z/WSl7ykespTnlI9/elPr/bee+9q//33rx7+8IdXT3va06rnPOc51T3vec/qwQ9+cLX77rsX0Gy//fbV4YcfXsp44hOfWFTjvve9bznuHve4R3XrW9+62nfffQt4HHu7292u1PuqV73qcgiaZuXQqnYRh8aiea0aSszmTI7EKMbEtkg1Z2Ivg2Mkpl/vetdblvYddtihAAQDsT5lYfc+++xTPfnJT66e8YxnVC9+8YurV77yldXLXvayAoAXvOAF1Qtf+MLqPve5T/XsZz+7evSjH1094AEPqB7/+MeXMh/1qEcVpeD0u93tbgUsj3jEI4pKUI/73//+RTX22muvUgbwON6xt7jFLZZ7G9PORK6Ew1vKGyf3zSHbrrGArvM5i+M4l5NJt9iNyRjFmJFrzPMdyfWOmb7HQP8ri1pc7WpXK8rhuOc973nVIYccUh111FHVK17xiuoxj3lMYa3zDj744PL7E57whOL8xz3ucUUVHvvYx5a6cDAQ3OUudykOBj7qAEzO8XINCvKwhz2s/AYge+yxRwGD4wG1r5czqTs8j+V2o0PAkDH7vgUSbaNnbfFe4sRZ4jQnk2kOF3sZnEMY1P+cgrVveMMbyv8MTCmw0ff+J/1e5FeZpJ9UO4azSf8jH/nI6vnPf365lu+ECTEegDhxt912K2DkOGHAtXxHER70oAdVT3rSk6pnPetZBUjYT2WUTwX8tt9++xWlUD/A0R4q1pzL6FrvuAZWCy9NXFs3ad1dl7TVE0KxWyJ397vfvcRXDsF+zPMCCEwi/95JOTYyKNaGYcKAxM45nISRSdK8c6ZEUo7A0cpNYshBwANkyqI8Qojwozx1wvKEIQCQQ7z85S8vL4CSO3C4awMENfCiJBQEeIQyQK8nhpMWs65k8te4VveoXtfkTNfgTnPyJX1lUq1bxdCydwbCQMbibKxj6AMOOKDIfqQe0xgXy575zGeW4zhOhi5uO95n4UEI4GSJI+fICXxHFbBejHYsEDhfAikMAAnwuB5Wu6brySMARc4gd5AzHHjggdVrXvOa8hswPPe5zy31UhfAAhq5iba6tlBnfKGrh7BGegNLE5drDVnW3bcwAyMZBpsjn4xLTsVzxgvjfc9p1AEYfHY8CQYG53O07zGUg8RiYUQ2LieQT1CAjCPoynk5xrmO4SQA9HItoUOZmOwa2P3Upz51Od77XgihAIAnLDheGJBIAi1QAaCQlLwF2DOy2DdbueqTQUNGnbqy/bC9rgZYT2L9RvYTozlNli02c+yLXvSiYtTDDjusGJXUyt4ZG+M5N8B47WtfW74j0ckXvMg241OXDOW6vncvrHd9ICT1ziXjQglgYTEQYnJyDN+rm++AhELoURx00EEFGByvXvIVjtY2vRWMB3ThhOoAhPyCGnTNMcw69jLX2cBJixq7kphmdivhM2hDYjGIkd7ylrcsx1kvUht553zMEkc5VdeNzPqesUkwwyuLZAOPcwMkjjWmkHl/AMggDnZyAPZzjpDAoRzomkClpyAkyDdcE0D8RvJd37UBguIIV44HFmDkeGAHQOULMX5TnnJcP2MXi1pCP0MeMGzp1hCQpI/vxfHkFzM4VTKGSdhHXr2wS3xlTPEV8+UEVIIScAYgcASQ6N6R5CSMyvWd4ziJlDM28Lk2o0s+JWQ+Gwp2HgWSkFIFDBaG1MF1XQfwvHOgeM+ZFAoI1NPvZF9OQeLFe+GHCiUc6Dp+4AMfWM4XhDvXoxBD1xeuSJLYt+Bh0vx/W/6AeQyS+I5pGCdhIuuYj+EklsMZFTC8Mx65znCt3sKb3vSm6sQTTyznvvSlLy0O42hsl8EDg7KogZ5CRhCxncMpjrIYXtwn75iaupNnx6in63E2IPifsqgvRTj66KOr17/+9UWJsFpoAipgl3Okx/D2t799OcQADSC4vnwGmNWpbxn8IheajL4xpF6pvnH9vDiFFJJFhuQQhuRcDiSh2ODzhz/84eqkk04qw7VknWMlaeIyB5NS8VbYMDyrL09+09dWPgBwts/YJwxIwBhZGUCgDpjsd4mm76iCkJD5h6iOurkmp7361a8uakOJEqKwX/0oSiatlOdcTHeu44UN52insrxTMeBT/wwWrcY6w1FDwXW2t8V88TYrahiEI8gig5JkBscahsF2zMgQ7Xve856S/GGwQR1xGFOUgcXUw+8f+tCHCvMZFVgcTyk4VF5BLThb3CW9mA6E6gEcQMLhypPpk2xKIHfQLRWX1TddOY4UUpLoqas6+911AYaCGG9QF8zWLnWTJAKqOgEPp7OB85X1ute9rlxDWFTPae6WWpH1AENiT0a6EvehWravwUmkMJEsYr7vvLAxzGI0TudMCqCrFSmXoBkFdD4DYj/j+s3LeWSbemT

### Multiple component fit

For multi-exponential models ($I=\sum_{i=1}^n a_i e^{-\frac{t}{\tau_i}}$), set `param.nComp` to the number of exponential components:

In [14]:
param.nComp = 2
// fitLMA automatically performs a RLF fit if no estimation is provided
// so the following step is redundant
// param.paramMap = rldRslt.paramMap
lmaRslt = op.run("flim.fitLMA", param)
// later fits shouldn't be affected
param.nComp = 1

// showing tau from 0.13 to 0.25
nb.display(fcd.tableDisp(lmaRslt, 0.13, 0.25, 900))

Z min = -67.75054931640625
Z max = 261.3349914550781
A1 min = -1523.2769775390625
A1 max = 1083.8193359375
Tau1 min = -1.0956814942208E13
Tau1 max = 1.6905548549062656E16
A2 min = -338.0961608886719
A2 max = 2269.291748046875
Tau2 min = -2.9590814988435456E16
Tau2 max = 4.596363691631163E24



Z A1 Tau1 Pseudocolor1 A2 Tau2 Pseudocolor2 <img src="data:image/png;charset=utf-8;base64,iVBORw0KGgoAAAANSUhEUgAAAIAAAACACAYAAADDPmHLAAAHD0lEQVR42u1dSW7DMAxMl3MvfUP/1v9/oIUPBgSCy1CWbUqeAjx0SZqIIy7DkfJ6vV5/GXt7e/vLPoZW2rgIBACNAGC4JwBoTwXA5+cnF4IRgEYA0AgAcgLn28fHByMAjQCgEQA0AiBp7+/vdBojAI0AoBEAGfv+/qazGQFoBACNAKA9DACkrAkAgulpANgctTtLOo0RgTUAdz8BwBriVgAwDDMCMBoQAHQIAVAcQA8AGHcB2iqyCyjS1zOtPAgAdzsj8/8nBc59C4ks2L7zWexNDIAznVcVGBPVELUKLesx1cSg1mvffr691onmD9csVHYBzl6wEc+/SFo6d4cfyd9Vc3/7mhaoT+4P81ftrtG6gJ7nix7z8/OzZgrIpAq5uFV32PalRYTovbELCOoDZJF+f3/Lik0mYxHPyYsZrt3bGVUW7mi4f0wXoDkzmwZmrbKrg3g4ALTctvfBSJivQrOOngjKqNbyAsvWAGdUxFcqe0d1K5OE/fMA4CF9jwyVdn/P/0ScLN9r4fQ2HgDWDkMKRS+PXrFo2XSAtn2FCaPzKv/2Z97u99oqdPGuoI61KJdNAQVrgXE7Rg5BquzaM6lfzaEeJb5EEagtgpfzM4tVWYW0f7+9Lw3sXiewbAqIdsNZE7grF1MDQdvuRiktSms3g2PsoljhbmQezyiJRg+odtvuV7Y4D/Q5iqSG80PmXXP6qJVDKGj5fvbQbzkQKYyLpYJait7RQg35v1sHah3KHSRSDygGXjc7t1bP24FycWW4jvIwKkOLIknx8fZ5I97R9C3iEBmiPZYy2s1IIacBKyr2ijGD4ydeo/N+RBQdSTcasKz2VQMg6kyUIFqiBhgFgGjHRspcDTwyd+5/h84pEIdn3k+BovCemXdmV2e7Di/8WwvvkTsZB/cAaMkuoHchPAdqenvrbxCWLtMR9EjAUEBMlwKQHIY6f3TbhxSKks1r0wAalr1+v3fsPXUKQBbaart6n1PuoC2/a1Std2pH/p0WARChZ3HVT52jYZlq3lpcK9dLB7c7vLXW6W0E0KIComewdv8IKdw0eoCRv7NyvbeQ2+7fTTpVC+/t93vksKJApq3zeISihNCYocyRGYA1WfOKPm0xPSdrUUFGiBZM2/d7y+gVhB5Ys3zEEoogj+mK2iqvZdMYP7nTW2fIsN861ivUZHrw5F3Zwnc5KjgzB0d68ywFq8mv9zSw/6wtDLWosu/yFhjee0D6/kz6KwCGc0SgUUvk7WyEKZNObR24O72NADLPyzTR/t4rKDMO9CLIUl2AtsO8XWflUmtcm5myyWq/dWab07V0sQPBagVRzV+kE1hCFJqlTDNsWaQs8gY6MpRru1ADm1YboFFP0sgZude0KcBzptxpmorG49+t3RONeqUT26KvfS1tikAo2YzA85Gng3skWNaIVe5QS8kjHauBoeUErKJQEj+Ro9q/k0CPiuGiwBivyrEKQI+zR1tDzXkSFBanL3+npYkj199YRS5a10x9OhhRxqDqGo/MkWHeiwayILQGQT2MX3QSeJIzAeedDczQoBZ1q9UImpBDI3jaMG3x/aiWEB1tW6AtXhNcc5IGDafaDtWGOVoIt0bBkva1ikv0ziKNkfQi4rIXRER8AKLK9WhWi4jRHCnBkaGfNUdpBSQ6A+lti28DwHbK5ajTtYIPORKlFUmeWsdT7GhzAakTQKjsrAMzQ7GlbgjJFnoeeJBizGPZtIpeRhyNk/BG0V43gZI+yymCeuVikTjTKhItYMjHaCHbI440x2eUwZqgpKflW4YJPCoItdQ91lm86NCIRc5EO9RqI1F6OBMFi7SH11+4hORJqe/zokR0TN1q+aKbTGQLaV2CUXh3X0sFI/fkZKTUVgWukS/RbCG7c732Dj0LsKQq+Kj4E6m+rXYRIWwi+tgq+KzHeCCI0tKjAZC9NAGVlXk6Qu9kD9KvWw5HiR1+ZhAYERDWzQurUv3jaf57oodGQ2tS8Ql3+/UAQG7iiIq7rP6wZ8oYpadMIUkAdMrLI3ZP4+AjISraFViFp3XKOKPze8wsoNc8AsaarKFRwnu8JJEyE82JiJ4aKaDnWDdyKRNye4in6ff0B5MXdzUjQOZqmej4drbD8CRlLfHUI/BkDTDgTj80KvRMHK3UgqiBJzvpU/t0cLZd9G7vQilntJtAgJe5PIIAGBgZeu4QQK58i6JP9iQUAXDiTGHERZERAbRIZT9vChh1lDp7w8dDnH0NAL6+vm5bzJ4Pdjh6fx+LwAms+mcUEQCFgLLAB0KvDYCzP0PwQSwgI4An+Mx2BwRA8SjwsNzOCEAjAGgEAI0AoBEALBQJABoBQCMAaAQAjQCgEQA0AoBGANC4AAQAjQCgEQA0AoBGANAIABoBQCMAaAQAjQCgEQA0AoBGANAIANqC9g/ynOHxHiqbLgAAAABJRU5ErkJggg==" /> <img src="data:image/png;charset=utf-8;base64,iVBORw0KGgoAAAANSUhEUgAAAIAAAACACAYAAADDPmHLAAAv8ElEQVR42u3d1Y90WxHG4Q93dz24u7u7u7u7u8PB3d3d3SUQIEjCDdwQuIA/pcmzwm+yT5+emR7rmflOT7LTPbvXXlL11lu1dJ84ceLEbNF1pjOdaeH9M5/5zLPNnlnm2unzm9VjN9d8XtP/97Ocg8jvAK8Ts5PhSuAHJfhjpND9BcBWVrNbQUmzKN1Wz27FHKtUzkkIhN3T9XbCOMtZznLgwtysjiexxR6+CzgIf7lW2CECgKVupYBllbMX97BTl3CYweNJxjYHI9gpNU+fWUTZ83mW5jAFewbqIew9k+18/VG65gG4G0CfZAHn3hQ/7X7thOJ3GtW7d7aznW2pXspOei4HQe/bPXMYYFrpOMB+xQzz11nPetZdlzNlqWXijd1a33Fiw6UBsJkw9hqksYR5a+j/eWUvO9iz6LdFFte9Y6iw/b0IYq++Zj991SKFLBM4TkGzyA2U7yp7Iuuh4BUEcQfpW3ei/GMMlIOz9EU0vpMuV9+XUezUPWyWHgvMd093Myx9hhoH2K1VzfvrnfSrl0m7jNBzb/VWznnOc57O9wPpTnoo+xn4HgngHEQltrOozYKy7SaDthr3T7kp1PcsPibyv66kaz7tFAxnP/vZ97VncNLGADulyWmAt1lgV55Zbc/M0/fUzfhtel3talcbn0BRGkrt8/znP//Gb/J0zwUY5z73ucd3vwcE9wPYtA4nSQ9i70Okux0smQo0BnCd5zzn2bDSC1zgAhsKCBAUc45znGN8pmiKu8hFLjLuee5c5zrXeFa6S1/60uO3i170oiPdFa5whdmVr3zl2eUvf/nZZS972Q1QSON/nxe72MVmF77whUc+/R4g3Fs0KLXuBWwTjE0t3GfWm+Vd8IIXnJ3vfOcbSiP8C13oQkOJ5z3veTeE76JwCqJYv3vm4Q9/+OxSl7rUUOpVr3rV2SUvecnZNa95zXEfI1zucpcb+d3xjncc5cnvlre85exWt7rV7EpXutKo6zWucY3xvDp5Vh4AcZnLXGaAR518v/rVrz67xCUuMcpeNtA90gBYdhHHfo6GZcmESvCEDQQEytoogEL9T9jSsjzgABT3UxgAnHLKKbN73/veo54UdNe73nX2vOc9b3bf+953dpe73GWU99CHPnT87/Nxj3vc7DrXuc7sZje72ezGN77xqBOQ3PCGNxxAuec97zn+v+IVrzi71rWuNbvd7W43lH/nO9951A2DKAe4APIYA+Fg1/TNA6LgjEJRLeVSPGumREK96U1vOrv1rW89FOx7VO0TCK53vettUDfLBACMcP3rX3/2mMc8ZiiIIvPtXMotbnGL2d3

In [15]:
// set # of exponential components
param.nComp = 3
globalRslt = op.run("flim.fitGlobal", param)
param.nComp = 1

// showing tau from 0.13 to 0.25
nb.display(fcd.tableDisp(globalRslt, 10, 10))

Z min = -5.333333492279053
Z max = 3.0639078617095947
A1 min = 0.0
A1 max = 273.18060302734375
Tau1 min = 0.0
Tau1 max = 1.1072466373443604
A2 min = 0.0
A2 max = 990.9171752929688
Tau2 min = 0.0
Tau2 max = 0.1581559181213379
A3 min = 0.0
A3 max = 958.3338623046875
Tau3 min = 0.0
Tau3 max = 0.15660516917705536
brightness_max automatically set to 1696.0



Z A1 Tau1 Pseudocolor1 A2 Tau2 Pseudocolor2 A3 Tau3 Pseudocolor3 <img src="data:image/png;charset=utf-8;base64,iVBORw0KGgoAAAANSUhEUgAAAIAAAACACAYAAADDPmHLAAAq8UlEQVR42u3dRYxsW9nG8Yt9uLu7u7u7u7u7u7sGh+AagjMgQUIgQJCEIGEGCWMmJEyZMCD95bfDv+86+1RVV/ep6tP30p3sVPWutZe87/Pqkn3GGWecsXPUrv/7v//bOYr9cp3znOc8sn074HX0OnWuc51rrXLnPve5l/523vOed+2y/+PX4TR0jnOc41CeGaV0XSAdtJ2zCbCOpWCuLbalsc6yADj/+c+/NQk4Fek7vo64BhiZe6qMXiZlezmM43NMwyacuLO4xK8HgHUl+3QT42zolf9v+ABphMzLqCHOc57znFT+ghe84Cm1twjQRzncPHYCz0bXETcZ+/OEF9nyVU7i/JpL914qfF3f4diZPOC1FwNi2PnOd76Nof4CF7jAgRhYXxaV1b95m2OfL3ShC+3261TDvlO5Fpm305x/2L4knW5H7eyqHTZkWvZPrLMiQc8KodsqDbdVH2AuoUeVWAdRe5tQudukx8UvfvHTZka2FgVA8EFt1Om0z/+jpuTgDDqIZIwO4DYIOndWDyq9257k2bAUHx4AIuim1OIyQuw3SVM9+3U4R0YvGpN7m2TWkdNwh+Whr8NQfRk1xDJijWUufOELT+V87jUXUX2VnT+T9lhU1xEL3TZp0o6mJ7ws9ev+CICLXOQiS3MLo6N12ctedmW7mE7aL3axi+0y6prXvOaeCa/9JME2YdIOxQSM0nrYMXwJmzkoXJe4xCVOYtr4/81vfvNdZGN4xLvWta61C6hxRtDzN7zhDXcuetGLrpX8usxlLjN9AgnAAYrv6zqN2wbLga7Rvs1tYEBoINsIh7SPIcvUY5JdH/zve/32GWFvfOMb71zykpfcud71rneSVqApPIfpyrh3xStecQLcpS51qen3q1zlKtOz9cVnNGjsygYk9V396lffpdVeTu06Tu+2zM8Kn+tg4U8EWaUhsrnzMotse4R1RdSxwze60Y2m73O7PgLB56Uvfempbxh5+9vffrp37Wtfe1d6/XaPe9xj5zrXuc7Ur8Dgd3UEmiQbqOYCQTPMaZIpWua3rGP/lz275VnLzav5VZpikbeuPDV91ate9QT7n3bw/brXve7E3LGeubnAFMwZ7b0yV77ylafvV7rSlU4gsvowzn3PAYE6rna1q02SeLnLXW7SCpW/1a1utQvQy1/+8jvXv/71p3KL/JBt2+6NzgVsMjRZZOd0dt5h6hfhEQ2TMQpRA8jNbnazE7SC/33e4AY3OMHuYtIqgujPFa5whZPua3u0/ZiY5lEXMOiTz+oGDN9H4N3iFreYnEzjyEcB4G3TdIMh6XadjKQC4UZGIFzfqWFMHYmWSnZhDMlUZnQE/UY9pt5JKylVN2BgajbVs6SS1Fd+WfTgdwDV32tc4xqTZsLUfAuAVbffbnrTm05labC5Kk+TzcF5xHIBp+ZELDMfyxweanOOaASPKD5JJgamul2IjYEIHwiEaaSRTY/Y1D+J1L7fyrVnXsZL2dFcAIe+jX0fzU4aKADpj3aXrW7KNKzrQK8KZU8rAJYxc90MWQOjxjEPCLKRpDY7Ojp/ESzpyhlLCusXszE6kZgQ05clh+585zvvtl1bwJYZ0hf/Ax0HMkfSPQAFPEAY+zmnkboCj7HuJ5u4SEPMQ9UNTlYdfhawBEuSOi9PWjEy7YKAgOC+7wgkfi/ki7jAlXagdgGA3Qa4iDg6herru3L+J93Mxe1ud7sJIL6P2oPvMGo94PZMJovmolX0cdQAR3FG8r91btdDHTuew7esMze5yU127nWve+2CYrSrpFEkMJcE4eHYnt8RHoAqW6gXGAGPtvE9zYGB1Ll2+pwD+H73u98JY0qDAJUL042vdtVzm9vcZurjKmduVTJpmbnd2oqgTU1yGGzExUiSirE5gosQTbrG+yMTMPGhD33oJLG3ve1tT0qcIE6xvb6QRG1R3SND3FMHv8E9Kj+TwaHzbOYpx7T/le352gQOz+m774BAmxi/9gJJU+SLGFf7+wnH19EIa4aiq9HV/aRyLw92XOCAaa6epUr7Tir8n5pELKqaGtVxfYiZ+Q9AZODKzaVmtPejWbnlLW+5+33UKBgVEZmTzID22G73aiu1jsH1F6j02YXZzBppNyYpaXR4+MMffgId0wD1dduTQ2uaoM3MUUd0g7vjHe+4mzBhuxEDUUmPQZMg5TEUcRGTecgBqx/uYwDikX7AAAR1xNgkrMFikt/nk0nqIanjfEOaIcYASPbb9+qmRfgW+p3for6igUDvuTvc4Q67YFcHreT/Bz/4wVOfEqi51AeGpHbb6wWG6Gd/dmcd1DZoDCZV4nLEAIAcrcrmZZeTHyMBTPV84MrBA6rsem0BXhpkJOg4Ddx4tFNkkjSSYECob9oeJWiMLGqTo+h7fTHe+9///lM/7373u+9GAre+9a2ne5m0iB+zV6n+MRm2qcUkM19k+T6+ZTakDox2CFNGf2CMaTEGCO52t7vt+gCkCRgMkNSyoSWIOIPV4/uoVRBUv0oe1d/73Oc+S9WfuvU5xhU6KqM9fXBhFKC6zxktEVX0EKN81x+MLJUMDMamvy972ct2nvjEJ059BSygbkxzxu53VdSmVlENQrF/ByPpms+UGSyJRTT3DJLDhmDl+kcPP2dK2WbXXAiaum4eYD43nwovLAOGmF7WsZBxP8vX5/MLefaj1GkTmAHGGPU1TXaXu9xlAnxzCjSKfrknS4kGADEKySanjZdph6U8nTe2l3qporl2MKCnPvWpuwP3v9k4gyYpDRYhqEPPFzJVZ3Ybw30+6lGP2mV8OX+MVXcx/xji1adR+8wzgKOmGkNKjGLze7ZIAsj1jzT7H/NzEpmFsf7REaVtxkwmOqszyS83sY0cwDrlB1N+4pz6fi+SxyEjAQgYo6hZRMWUcbCjD9Fsn9/HZBEQxXh1GBBQjWnb6ixUS12PcT9AjaqWd66f2XMAcG+UQBogTVS0oi0ay9jK7j3pSU/anW7GaPY/GhS10AZjFAIg7mkfEJmFZz/72SfQUxuHtWD0v9pu/6pj7mFHUNKBEKSiT5LSggtE9SlcKs73bLE1EPKW88aBYGTGPAxFUIAa8/2YhQk5c2NfW/Y193cwNWAmtfpJivMTlAGGwGWs2jIGTAV4v2Xr9T+BGPMZtBba+G3UXqdjveB/eX6wVaslNkrTNlFiwEkNJpDsHLj5gg/qdkzUJNHZYcDI81dXBCvFm9bwPF8DEzBtjDKqB0MyN9pIkrPd49iNAagwnp33fJpN31/60pfuMvphD3vYBPCc2PwAfSwDmFZAq+6N4w2Q0fO0TgaNzCiEGtHp3jgBojxiPvrRj959tkydcggHDHe9611PUM9JWqqeF52XnZfuElbVHoZ4jlQHOs/kT7ivvdHZhHLMbJZRXwCsNYOeqz1MC9CjlmAm1NmCkCaF9Jm/QsIzoYWZ2iqhNc4/xGDPlD2ca+FDPrdg/XTiePpWEoGQY3oX2g02R+5Od7rTdOUgtepGGZJSWc/e+973nj7TAOX51YkpGAxA9Uk5vxdFYDTCFx6ODl8Sr6+eK5cw+hXqJ/Hqrw6MSgCy337HJNrDVfRBK7Rw5SUveckuqPgKtGDZTc8HinmEcRo2lBzswRIvmPnkJz95khCDf8QjHjERKnVazl2o1Hy+3zBCsmRcKOJ39pFWUL51AX4TdjEZ2ozJmJj0Us0tGBm3gmNgi0/Uq98uzMaAzEdOJWlnXlp86p7p49GOK+t39XXfuNCAVOszX6bZSpoCOPTtOc95zi7otGOa27gzdYe5WOQEJ3BZw4sQlzQheAwoz04i+AIYhsAPeMADJqJQn4hEwiLAGFI2j4+wOlYcrywGkCjPj

### Phasor Analysis

In [16]:
// WIP
param.paramMap = null
phasorRslt = op.run("flim.fitPhasor", param)

flimlib.flimj.FitResults@3020aa63

## Other settings

### Region of Interest

Sometimes, instead of the whole dataset, only part of the image (e.g. the region near the nucleus) are of our interest. By specifying the `roi` parameter, we neglect unwanted parts outside of it during fitting. This greatly improves the running time on large images.

In [17]:
import net.imglib2.roi.geom.real.OpenWritableBox

min = [ 20, 20 ]
max = [ 100, 100 ]

// define our region of interest, in this case [40, 87] * [40, 87]
roi = new OpenWritableBox([ min[0] - 1, min[1] - 1 ] as double[], [ max[0] + 1, max[1] + 1 ] as double[])

net.imglib2.roi.geom.real.OpenWritableBox@1d29

We start the fitting routine the same way as before but with the `roi` parameter:

In [18]:
// fitLMA with roi
lmaRslt = op.run("flim.fitLMA", param, roi)
nb.display(fcd.tableDisp(lmaRslt, 0.13, 0.25))

Z min = -0.23343004286289215
Z max = 5.321786880493164
A1 min = 0.0
A1 max = 1088.684814453125
Tau1 min = 0.0
Tau1 max = 3.1886990070343018
brightness_max automatically set to 1696.0



Z A1 Tau1 Pseudocolor1 <img src="data:image/png;charset=utf-8;base64,iVBORw0KGgoAAAANSUhEUgAAAIAAAACACAYAAADDPmHLAAAlsElEQVR42u3d57JtVZnGcYKoCEpQAREUkCgioIiSkaSggGJAJUkSRBCUIEoWRFAxgBhIonwgJ0vFoqjig1yDN0D5gauYXb9R/T+1yj7Ybfem91mHuatmzbVnGOF9njeMODfYYIMNpvl4XR+zEGYCzMdMgPmYCTAfMwHmYybAfMwEmI+ZAPMxE2A+ZgLMx0yA+ZgJMB8zAeZjJsB8zASYj5kA8zETYD5mAszHTID5mAnwvz7e+MY3Tm9605vG73e84x3Te9/73mmrrbaa3vnOd457m2+++bTllltO22233bTjjjuOZ1zfbLPNxnP777//9La3vW3aaKONxjubbLLJ9OY3v3k6+OCDp5122mnac889p/3222865phjxrPHH3/8dMABB0yf+MQnxvld73rXtMUWW0w77LDDOD74wQ+OvHbbbbdp9913n7bZZpvp7W9/+0jT7/e9733TzjvvPG299dbjfXnL1/Ouv/Wtbx3lU47XPQE23HDD/3Jt4403XvMbaATpGnABTdCuAQ4xAA5o9wEDVL9dAx6CbLvttgOIt7zlLeMAwi677DLtu+++4/zJT35yOvzww0f6Rx111AB0jz32mLbffvvp6KOPHuB57kMf+tDIF/Ce9Yw8pYeY7jnvtddeo/zKAWjpvOc97xnvIIHyqNsb3vCGtcplHSbHyiaooouAr40M7373uwfgzgClTc6ECyha5wC8590jbEJf1FLaG+EiRmnSWs96L1ARwW/aL+2DDjpo3PdOZEMI+QDdIQ1llRft9y6LIZ0DDzxwkAgJdt1111F2pFQeZZldwH8etJpGZfaBRvCAJOC99957aNAHPvCBIUTH+9///jVEIXwa6BmCRhgAA4OpRxLpe8+9rIL3WBPXAO4ZINNSrgHQtBZoyvCRj3xk/EYG4Lsvz49+9KMjbdYlAvrfPe99+tOfHuXznrRd33TTTdfU939ysILrJQESAo0Ahv8BQov49A9/+MPDzNIigBM80JlV/wPOQcC0juZzBTQPSbwPFKTyjmtAAYZ05At4oLAMnpEHn03o0kIkzyDXoYceOt5FHmRBCmSRP+Ajn3S4GPmpizJI0ztIpK6Bujbzj3jrtQUAbr9pBY1MGwFJgMhASB/72MeGdvkfGAACMIE7AOp5YCOFtGkn7Rf4CfJcI2hHQZ78kCfyAQfRaK/ngSwfv4HHAskHwNJHDnnm86UlT2ktuiUEBjoSSd97nnHfc9XzvwN9lazAa5PwYmUJl0mkWQQIUJrjf+YVyIRI+2gTP0wbaVXaz9/ywd5xjXCBJ9L3Pw0n/MAGGhL4H9DOgEegffbZZ6SjHP6XNxIy5fL3vrNyybvYIyvief8jtWfUkWWSv3xdV37vKYO6I8XrLgYAvIrTbhqSuSVUWk7Q+dUCviJzIBVsMcGue8dvQLpPWw877LABJi0j/IIwacn/5JNPHmC4xiU4kE9+NB4BAScNh3yOO+646ZBDDlnjYryvHJ6XPwuhLLUapIkcDmVCEtYGuViF4p9/FRyvVwQAAp8McEKhgcAgUMIjIEcBEzPuOsAJO60EBkEiypFHHrlGG6Vd80skzzKwCIumntCVg/a5RvhMvfQd0uaOaC5S8vXe01fgXWXyfgc3wwrJxzvFDMB3X/1cl6ayeU6+LASyuYcEyvPP8lrbtaUmAGABSlNVGtg0hzYBGXA0j0YBkBVwP21Oe1xHBNdYBSABOx9OwPI59thjBxCsQtG7QxpF9vnmArjKKi/ERAKAImSEkpbyIpZrXFSuC2h8tjOQI3ZNUelKC2mV1//yQ5x/lpeyrTcEUMF6yWpLA9mZEIHO9xLMl770pSE4ByuR9gCexgBVh87nP//5AQgwpUuYwEQC/hbhCrj87z2AAghJagkAvl7GRZdUi4Sm5moADVhldhYjFFR6TxoFf56XfhaO9VAeHU61GpRNHsr6r1oHS08AQDKVBK2ZBEyBH+0hMJpP4F/84hfHNc8DmzDywQRFszwnMCNIzyMPSwBg1oQgCZ9waR0znEmmdXUvM/8sQi0Pz3gWmPICnDNQlYclQDL+XbpIgHh1PknLO/U71KnlOVYssiK+OtVkRE518L53BcrKsV4RgNAIMH8IHBqR9hftEwyzjyCEQMB+05ZcAOK4RoDOhO6ox45GIhQtFiMgROY0Uyttv4HkHjLUOkA6+dDKWgrKV6eRsrte30EEcl8ZPvWpT41ruSnXlQMJWC0kVR//I7ayel6a8l0vLQACAKs+fGeazOTTCoKMJHx35pRWAED0Xc8gAQPa/xdccMEgRKZburQIeLUeaDFwgU7bar8joryVgWVxrc4phPIuEN3PPQASqMrhft3H8lV2ZStt9WX1aDmyX3XVVaMloYlak1G96k4uPljbuMnSE4BA66MndKC5RkCEQ2AqXwcKM1gbGYBAAQBBsRD139efT+vTzoKsYgL5Asr7uQT36tMvokeMOnQaM5BWXcbOjgadpFkz0nX5cEsAZu4RQPmOOOKIUU9puuYAPCJ5rm5kloE1lN46QIKVTVAkDnBnAmL+PvOZzwxBBCphLLaZHTSP1jLj/gfyZz/72fEeImXy64ihhb2f2Qes34RcpO4ZaRawAd51xPAukkgLSZCtvGtOuq+s7tdEBZ46us7PAx2YTLzfngOycrMeCFHTU/7eJx8K8Wqjh0tLAKADgdlWwY9//OPTKaecMjSUljTQQosa6KFVBOQ3EhASDQMaLaLltSRYCe8hEpCAIN7w2+E3EGte1clDi0Xy7jnLz3VpA0MZvC99fh5xgFOzs5G/hocLToEpDeCzHsD2DMJyfeQg5kFkaXzuc58bSqFcnieTxS5gZVh6CwBIwCGAg8BPPfXUIVS+tUje//X4AZiGOosNvvKVrwywaJqAqjEC7zZpJAtAK/3PrSAGUiBc8wcAkkbnJqTbUHHpsCAAbHgZ4NLOrdWicchH2Vkb4BbcIbz81AP4dWHLi1UAvuueUb6alatoCVY2QUADUMVOOumkIQAEoGEqnSAQQ6D0ne98Z9wXQKWN3ic0ggY4YRMoYTckLH2a2Xh8gy/chnyaU+AZ7y42CxsVBC4CSJMWIpGAUnr1VzjXeSR/ZJWHszoAtN4+75144omjvP5HBofn1EO9lUm9CibVV77FAv/OEPI6SQBCq/kDWP6f5vpN6xe1ign9xje+MZ5HgJpcDf4Q5plnnjnue76AkqYDDlgObqOmYm5BHnX1AoN253KyJo0r+O1Zwm+6l2eB5Cxf9RD5NzagDA0RywuA3If/3XNGZOAjoPgCqRtsajqasnpPPVapWbiyCTLxBKWXT8Uvuuii6etf//r4XfOJEBz61wVKhMOMih+cC+L4zsVOGJpLIwHlAEC9eQGfL68foujdczQNQf1mLWo5NF5R7yJAHfJAgnrzmmEEYIGicjZy6XdAq4eOK5ZIeZGHAmglKKt61CRuDKN5jaswLrCyCZ5xxhnTNddcsybivfDCC0dlCQQ5CAI4J5xwwvTlL395aIjrBEyITCLtJyzBY9pSUNW0MYJswMa7ngMQYFgE4Dm7V5POmaWoqVaTzv/SA0J5KaP3EeK0004bhKiZywqoG3dXnsy/64JBlo2b8L/6/uAHPxjuTp1ZtHoHPUs28kEqdVC3pe8I+upXvzqEdvrpp49OEVpc8CcKJjxnABMMzXHPwRU0whdI/CiLQliZX8Bn6gvYgAZoWptVaK5h8wGkpyyNIXjGfeSSBo0UZJaue8iIrMrEgmQRmrsgHWfazuyzbPJCevU855xzpvPPP39NBxjLJV9pNVzc5JSldwEqr4kESOAigQrTlOuuu264APcJhDBosWeBSwj8Nd/ckC6r4L50CRkggG5WUDN3mE9Cb6IIAecimi/IKiDC4pgDoJlsYDD

In the results above, all other regions outside the box is neglected.

### Binning

Binning settings are enabled by setting the binning kernel parameter. The kernel can be any image. Here we use the built-in `SQUARE_KERNEL_3`, a $3\times3$ image with each pixel valued $\frac{1}{9}$:

In [19]:
import flimlib.flimj.FlimOps
FlimOps.SQUARE_KERNEL_3

In [20]:
import flimlib.flimj.FlimOps

// spin!
lmaRslt = ij.op().run("flim.fitLMA", param, roi, FlimOps.SQUARE_KERNEL_3)

nb.display(fcd.tableDisp(lmaRslt, 0.13, 0.25))

Z min = -0.42977118492126465
Z max = 26.733793258666992
A1 min = 0.0
A1 max = 8187.78955078125
Tau1 min = 0.0
Tau1 max = 1.191415548324585
brightness_max automatically set to 13512.0



Z A1 Tau1 Pseudocolor1 <img src="data:image/png;charset=utf-8;base64,iVBORw0KGgoAAAANSUhEUgAAAIAAAACACAYAAADDPmHLAAAlSUlEQVR42u3dY8x3y73G8V3bNndt27Zt27Ztt7veuzZT20qd2k76vu+m+Uzy3Zk82e05Oe/Wc9adTNb/XpyZ6/phfqNDDjnkkLGn/9dpr4SdAHvaCbCnnQB72gmwp50Ae9oJsKedAHvaCbCnnQB72gmwp50Ae9oJsKedAHvaCbCnnQB72gmwp50Ae9oJsKedAP+ndIxjHGMc+9jHHsc5znHG8Y9//HGCE5xgnPzkJ5/pFKc4xTjlKU85TnOa08zj6U9/+nk885nPPE53utPNdKYznWmmM57xjONUpzrVfObEJz7xOOtZzzoOPfTQceELX3hc/OIXH1e72tXGVa5ylXH1q199XPrSlx6Xv/zlx0UucpH521G64AUvOC50oQvN+33jPOc5zzz67qlPfer5+yxnOcv8nnPnPOc553ecKy9nOMMZ5rWTnexkszzKdsxjHvMoyy0d/ehH/6/1c7SjHe3gJYDCAR5gJznJSWalSQBfk0oFsIo+29nONs5+9rNPcM9xjnPM384Bx72eP+lJTzqfOde5zjUJcJnLXGYS4KpXveokwSUveclxpStdaVzucpebhHCPcwjg98UudrH57ote9KLj3Oc+9wTaN3zT/5LfCOIb5zvf+eY55HHuvOc97/w+Mh7veMebZUSCwHSUgO+ICP8JaATaNAEq7IHnFV7Bk/ik57SnPe08kiwgOA94Fa6yAeEIHFILHJXvGfeS1DQBUrgfMECVAAwkx8AGoAQ475b8Pv/5zz/v9Q3Je5BECnTXI4JnIpHr8oOQJzrRiaY2SBNUHxHguMc97qwLx7XenDsq7bE5Ahyo5hSKVCiwyqEyVaYKVNEB5H8kQIYky32AAT4SOI8cNAEQUsmIFFk84z1SYPodkO6LgDQN4nin7yCI72YapEtc4hJHagxJHpy77GUvO654xSuOS13qUvMZ74yMNBxtQKJT/RGh/9d6qt7+JxOxOROgYMc61rGm5KsUEgu8gAeWSlehF7jABY605cAAAsBUuspGgFXtek7lI493AtRRypYjht+SbyKL+wDFjjMj7vNO7/ctJJIX4Hs/k4KAvo0Ivi1fmRv3ZQa8F8mRgGmSViKs2mAF22/15J6DggCrSlMBKkQFAUHlqjAV6Df7fIUrXGFWbioWAVxP0iQOnHOpXf971j1AI81phxxHBMis0ALeKw/uy2RIzvMRgAlo/wMXyTiQfTez4Fj+pbRQzqyySgjG54kE6gPYB6p69eXc5k3AgYwm+ewiyVchJJ70BLLKVOlJXw4aMAAMANcjQGrcfYC58pWvPO8HqhYDCQRCEp7j6AjwnLzAS5q9m9PoO6Qeqbzf9zM9tIDzziFQ2sT7aikoZy0GCSlpAfWwaoGkf/WXjooYmyTA2sQDBClQQQBQ4aWabCo2dQ4A2gAgKtt1/1/jGteYR88BA6jucb9ngVHzEeFIIYkEQuZAHpJc7/YbIXwD4a5//etP0CMC++4eBHA/jeA34iIU8H3Pe5EtqadRVh8HCdx3whOecNbNquZzjg8kxmYJoCAKyt4rtMIDC2gkLdWeR00KOxcRJOAiBHBIuOYcYGgAfoL3Asd97vE+JHA+KcyWk9bMgPuSaNcAWd58R0IC30a4HFCE8H3XAFssALlqSdRE9T1JXn3DN5FRnXCEacbVQS4u8p9aUJsiAMkHACBUbiBUoanXvOscu4B3PgfQ/yqemr/uda87rne9602AVLJvIELNtxy8NTjjer4ATeQaMOSp86nrzErefw6evKdpaAT59d7su/coZ8/Kc/4DssivZ5BDHvIHAB4BEEI6KFoBAZ9U5URd61rXGte85jWn0yYY439qXcWqZNKi4gLTOWrZfTe/+c3HTW9603HjG994vktF11oofuCYmZH8VunZZmCXt6QXgDmE8uqao2/Ld04gAqQdyt9KssyIvCGJpKzyr3y1XBDPd2lGRCAsiJD3nznYNAFUYG12R4CLyl3nOteZKlRSOewtqU7N1rTzuyCQiif1wHc/4tQ2L3aQCgaI3yo3iS9SyB8A1noPlZwKr0maRw+wHL6agjUPkQexIkotBvenqRD22te+9iyrc6uvooyZKSRgEmoZHBQEyKOvOQVgBFAhJN5vQN7sZjcbN7rRjSZBVGaeeSZBci8C3PKWtxw3uMEN5jsCR0U6FjQqMqgFAHigF5TJUy+EXCshjZD69z6glo++k9koNL2aNFqiWAGS3/rWtx73v//9xz3vec+Z77Sda8jgnQhe/0E+QU7g5n0Aaq8QrEqkDtMAKgPgOVQqEFFIVECSJudUaIRRgcUJAFGEDxmK/CXRKwFIes3PTIT/u9dzAC00XEi3AJT8FGlEHO/xbYAj5A1veMNZHsAqz61udatx+9vfftz97ncft73tbacmUGZloPXygbxbvuWFw3xggGjTBFBgLFdQlUMlUt+pcHay3jn3qOwqV0XXoaNCaQjq39GzVHLgZzIK66pMEgX07HNOIVLVqVRgqJhETiqyFo7OXudjFCqOdEhOM0lIuvopd7vb3SYRbnGLWxwp9YQAoYsx+F30Un7Xtv/mNUCROYVUQbe5zW1mZSDAGs51zPECRk6ZymZvVSzgEUiqdVAzKxuclCJA0g98ybtyFmuR1CSt63jtcwDwGkb2rFQ4ug4lZWSOqPuI7f/ATiOQes3XYhiRxfl6Jb2fFqivYPOBIIVTCQqugqjCO97xjtODV3FpgDWmrnLrdkWOpN8zwFehOVAAKoZQx0xNOja9jp9aAWu7HIFS7ZHIuaTec0XzaIq1TwChJeQOYKAjAGCl7gv0NJf7lbtzNEX3F8GsNbB5AqgQwDlSi9ShAlOHKockFN/3WwXRFNRqZsPzNIdjlYswSXx2v4hgQOc/ALaOoFoUPVvgCXnWaKB7aQ9apE4l30RWAMorEB1vcpObTAIgJ8KT6uIbyK+sys/R5Su4p/uV1fWinZlAWkB8QNo0AWqvK7h0hzvcYWoB0qJyFFrhAa0ib3e724273OUuR0qS+zyv8njRkUcF11/Ax8hGZ59JMYIVrAFqnUO19albRKkfH8DI4lpNs0YC1dsIdE5c+WXO5JOUS8BPnUudA3rNXf+nNTxb/0URSd8uQCQ2sGkC8IxJvUSK73rXu04TEAFqPqmUO93pTuM+97nP9JrdD2gVlPOICAhAatZOGlKTvS72riKdd91v5/L+A56KBzYC1UppDAKTkf/g/4JBDS8DGkCVTzloLMn5glnIogzuyfFVBpqQOVS2hqTVo4iozBcNAPzNEwDogFVoTaI73/nOE2SVwv6pPBVBOyCHNrMjaQd6EULXI0ASk8oEWqodWDXlgNDInMbz1fZ3zv2RBPhFIB3r0WtQSQ4jEtQcbYgZTRPwgc+syJ97lT3gacB73/ves3VAg+QH1cGUD9PQNmMJNk0A6vx+97vfuMc97jEe8pCHjAc96EHjoQ996JGVkYZgFlQK8CMHSQc8EiADacp+AkAC4KrWJaCtar4BpFSra3USqehG/QAg01Hbv+Fg64gixJMX+aLqCwkjcna8IJIyILw6QH7lfcQjHjEe+chHjgc84AGT5JWD9vC85+SRD1JfwaYJgO2iYMBV6Pve977zHBVIE6gcKp9ZUFkqBfBMBLCB7rdzpEzymzp1nbTl2QOrWECOYT1/9RE4l6lAgrqhSb7zdUnXzHS98C+SAIpUlx/S3+BS/kBgekYelQvxlfvhD3/4eNKTnjQe+9jHzv/5O4hUs1g5EajmLdLqMdw0AYBLqpEA4AqepD/wgQ+c/9MQKgMpkq5A9nuVkELBkusqeiVAqrpBmoWVG+kjseukq+7p4gJ+N74vu44QflPT/kcUGoPqR8BC2bQSlV4wyHXA0nw0HhI86lGPGo973OPm/zQCc+a